### Import related package

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

## Initializing module
from sklearn.linear_model import LinearRegression
np.set_printoptions(suppress=True)

## Copy module
import copy

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


### Control memory usage space for GPU

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.3)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.compat.v1.keras.backend.set_session(sess)

### Preprocessing the data

In [3]:
def read(path):
    return pd.read_csv(path)

In [4]:
def buildTrain(train, pastWeek=4, futureWeek=4, defaultWeek=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureWeek-pastWeek):
        X = np.array(train.iloc[i:i+defaultWeek])
        X = np.append(X,train["CCSP"].iloc[i+defaultWeek:i+pastWeek])
        X_train.append(X.reshape(X.size))
        Y_train.append(np.array(train.iloc[i+pastWeek:i+pastWeek+futureWeek]["CCSP"]))
    return np.array(X_train), np.array(Y_train)

### Min-max normalization

In [5]:
sc = MinMaxScaler(feature_range = (0, 1))

In [6]:
path = "WeeklyFinalData.csv"
data = read(path)

date = data["Date"]
data.drop("Date", axis=1, inplace=True)
x_data, y_data = buildTrain(data, futureWeek=4)


a = sc.fit_transform(y_data[:,1].reshape(-1,1))
2000/(sc.data_max_-sc.data_min_)

array([0.07812195])

### Design get_data() to get data

In [7]:
def get_data(futureWeek):
    
    ## Read weekly copper price data
    path = "WeeklyFinalData.csv"
    data = read(path)
    
    date = data["Date"]
    data.drop("Date", axis=1, inplace=True)
    
    ## Add time lag (pastWeek=4, futureWeek=1)
    x_data, y_data = buildTrain(data, futureWeek=futureWeek)
    
    x_train_data = x_data[:int(x_data.shape[0]*0.8)]
    x_test_data = x_data[int(x_data.shape[0]*0.8):]
    y_train_data = y_data[:int(x_data.shape[0]*0.8)]
    y_test_data = y_data[int(x_data.shape[0]*0.8):]
#     ## Data split
#     x_train = x_data[0:int(x_data.shape[0]*0.8)]
#     x_test = x_data[int(x_data.shape[0]*0.8):]
    
#     y_train = y_data[0:int(y_data.shape[0]*0.8)]
#     y_test = y_data[int(y_data.shape[0]*0.8):]
    
    ## Normalize
#     x_train_scaled = sc.fit_transform(x_data)
#     y_train_scaled = sc.fit_transform(y_data)
    
#     x_train_scaled = x_data
#     y_train_scaled = y_data
    
#     return (x_train_scaled, y_train_scaled)

    return (x_train_data, x_test_data, y_train_data, y_test_data)

In [8]:
x_train_data, x_test_data, y_train_data, y_test_data = get_data(4)

print(x_train_data.shape)
print(x_test_data.shape)
print(y_train_data.shape)
print(y_test_data.shape)

(377, 18)
(95, 18)
(377, 4)
(95, 4)


### Network class

In [9]:
class Network():
    
    def __init__(self, nb_neuro, x_train_scaled, y_train_scaled):
        
#         x_train_scaled, y_train_scaled = get_data(nb_neuro)
        
        # Stop criteria - threshold
        self.threshold_for_error = 0.07
        self.threshold_for_lr = 1e-6
        
        # Input data
        self.x = tf.convert_to_tensor(x_train_scaled, np.float32)
        self.y = tf.convert_to_tensor(y_train_scaled, np.float32)
        
        # Learning rate
        self.learning_rate = 1e-2
        
        # Optimizer
#         self.optimizer = tf.optimizers.SGD(self.learning_rate)
        
         # Hidden layer I
        self.n_neurons_in_h1 = nb_neuro
        self.W1 = tf.Variable(tf.random.truncated_normal([self.x.shape[1], self.n_neurons_in_h1], mean=0, stddev=1))
        self.b1 = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1], mean=0, stddev=1))

        # Output layer
        self.Wo = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1, self.y.shape[1]], mean=0, stddev=1))
        self.bo = tf.Variable(tf.random.truncated_normal([self.y.shape[1]], mean=0, stddev=1))

        # Whether the network is acceptable
        self.acceptable = False
    
    def forecast(self, x_test_scaled):
    
        x_test_scaled = tf.cast(x_test_scaled, tf.float32)
        activation_value = tf.nn.relu((tf.matmul(x_test_scaled, self.W1)+self.b1))
        forecast_value = tf.matmul(activation_value,self.Wo)+self.bo
       
        return forecast_value

    def setData(self, x_train_scaled, y_train_scaled):
        self.x = tf.convert_to_tensor(x_train_scaled, np.float32)
        self.y = tf.convert_to_tensor(y_train_scaled, np.float32)
    
    def addData(self, new_x_train, new_y_train):
#         self.x = tf.convert_to_tensor(x_train_scaled, np.float32)
#         self.y = tf.convert_to_tensor(y_train_scaled, np.float32)
        self.x = tf.concat([self.x, new_x_train.reshape(1,-1)],0)
        self.y = tf.concat([self.y, new_y_train.reshape(1,-1)],0)
    
        # forward operation
    def forward(self,  reg_strength= 0):
        with tf.GradientTape() as tape:

            y1 = tf.nn.relu((tf.matmul(self.x, self.W1)+self.b1))
            yo = (tf.matmul(y1,self.Wo)+self.bo)

            # performance measure
            diff = yo-self.y
            loss = tf.reduce_mean(diff**2) + (reg_strength/(self.Wo.shape[1]*(self.Wo.shape[0]+1)+self.W1.shape[1]*(self.W1.shape[0]+1))) * ((tf.nn.l2_loss(self.W1) + tf.nn.l2_loss(self.Wo) + tf.nn.l2_loss(self.b1) + tf.nn.l2_loss(self.bo))*2)
#             loss = tf.reduce_mean(diff**2, axis=0) + reg_strength * (tf.nn.l2_loss(self.W1) + tf.nn.l2_loss(self.Wo) + tf.nn.l2_loss(self.b1) + tf.nn.l2_loss(self.bo))

        return(yo, loss, tape)

    # backward operation
    def backward_Adam(self,tape,loss):

#         tape.watch([self.W1, self.Wo, self.b1, self.bo])
        optimizer = tf.optimizers.Adam(self.learning_rate)
        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])
        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))
    
    def backward_RMS(self,tape,loss):

        optimizer = tf.keras.optimizers.RMSprop(self.learning_rate)
        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])
        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))

### Matching module (Check)

In [10]:
# tunning the parameter
def matching(network):

    network.learning_rate = 1e-3
    
    while True:
        
        yo, loss, tape = network.forward()

        if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
            
            network.acceptable = True
            print("Matching finished - the network is acceptable")
            return(network)


        else:
            
            # Save the current papameter
            network_pre = copy.deepcopy(network)
            loss_pre = loss
            
            # tuning and check the loss performance of the next step
            network.backward_Adam(tape,loss)
            yo, loss, tape = network.forward()

            # Confirm whether the adjusted loss value is smaller than the current one
            if loss < loss_pre:

                # Multiply the learning rate by 1.2
                network.learning_rate *= 1.2

            # On the contrary, reduce the learning rate
            else:

                network = network_pre
                
                # Identify whether the current learning rate is less than the threshold
                if network.learning_rate <= network.threshold_for_lr:
                    network.acceptable = False
                    # If true, return the current model parameters
                    print("Matching finished - the network is Unacceptable")
                    return(network)

                # On the contrary, maintain the original parameter and adjust the learning rate
                else:
                    network.learning_rate *= 0.7
#                     print("B",network.learning_rate)

### Initializing module (Check)

In [11]:
def initializing(network, initial_x, initial_y):
    
#     x_train_scaled, y_train_scaled = get_data(4)

#     initial_x = x_train_scaled[:x_train_scaled.shape[1]+1]
#     initial_y = y_train_scaled[:x_train_scaled.shape[1]+1]
    min_y = tf.reduce_min(initial_y, axis=0)
    res_y = initial_y-min_y
    reg = LinearRegression().fit(initial_x, res_y)

    network.W1 = tf.Variable(tf.cast(tf.transpose(reg.coef_), tf.float32))
    network.b1 = tf.Variable(tf.convert_to_tensor(reg.intercept_, tf.float32))
    network.Wo = tf.Variable(tf.convert_to_tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], tf.float32))
    network.bo = tf.Variable(tf.cast(min_y, tf.float32))

    network.acceptable =True
#     network.W1 = tf.cast(tf.transpose(reg.coef_), tf.float32)
#     network.b1 = tf.convert_to_tensor(reg.intercept_, tf.float32)
#     network.Wo = tf.convert_to_tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], tf.float32)
#     network.bo = tf.cast(min_y, tf.float32)

### Selecting module (Check)

In [12]:
def selecting(network, x_train_scaled, y_train_scaled):
    
    loss = []
    temp_network = copy.deepcopy(network)
    
    for i in range(x_train_scaled.shape[0]):
        temp_network.setData(x_train_scaled[i].reshape(1,-1), y_train_scaled[i].reshape(1,-1))
        loss.append((temp_network.forward()[1].numpy(),i))

    sorted_index = [sorted_data[1] for sorted_data in sorted(loss, key = lambda x:x[0])]
#     sorted_index = [x[1] for x in sorted(loss, key = lambda x:sum(x[0]))]
#     sorted_index = [x for x in sorted(loss, key = lambda x:x[0])]

    
    print("First:",loss[sorted_index[0]])
   # print("Second:",loss[sorted_index[1]])
    print("Selecting module finish!")
    
    return sorted_index

### Regularizing module (Check)

In [13]:
def regularizing(network):

    
    network.learning_rate = 1e-3

    for i in range(100):

            
#         W1_pre, b1_pre, Wo_pre, bo_pre = network.W1, network.b1, network.Wo, network.bo
        network_pre = copy.deepcopy(network)
        yo, loss, tape = network.forward(1e-2)

        loss_pre = loss

        network.backward_Adam(tape, loss)
        yo, loss, tape = network.forward(1e-2)

        if loss <= loss_pre:
            if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
                network.learning_rate *= 1.2
                print("Regularizing process")

            else:
#                 network.W1, network.b1, network.Wo, network.bo = W1_pre, b1_pre, Wo_pre, bo_pre
                network = network_pre

                print("Regularizing finished(A)")
                return(network)
#                     break

        else:


            network = network_pre

            if network.learning_rate > network.threshold_for_lr:
                network.learning_rate *= 0.7

            else:

                print("Regularizing finished(B)")
                return(network)
#                     break

        if i == 99:
            
            return(network)

### Reorganizing module (Check)

In [14]:
def reorganizing(network):
    
#     if network.acceptable:
        
    k = 1
    p = network.W1.shape[1]

    while True:



        if k > p:

            print("The number of neuro: ",p)
            return(network)

        else:


            network = regularizing(network)
            network_pre = copy.deepcopy(network)

            network.acceptable = False

            network.W1 = tf.Variable(tf.concat([network.W1[:,:k-1],network.W1[:,k:]],1))
            network.b1 = tf.Variable(tf.concat([network.b1[:k-1],network.b1[k:]],0))
            network.Wo = tf.Variable(tf.concat([network.Wo[:k-1,:],network.Wo[k:,:]],0))

#             print(network.W1.shape, network.Wo.shape, network.b1.shape)
            network = matching(network)

            if network.acceptable:

                print("Drop out the nero number: %d / %d" %(k, p))
                p-=1


            else:
                network = network_pre
                print("Cannot drop out the nero number: %d / %d" %(k, p))
                k+=1
                    
#     else:
#         return None

### Cramming module (Check)

In [15]:
def cramming(network):
    
    ## Test network
# x_train_scaled, y_train_scaled = get_data(4)
# initial_x = x_train_scaled[:x_train_scaled.shape[1]+1]
# initial_y = y_train_scaled[:x_train_scaled.shape[1]+1]

# network = Network(4, initial_x, initial_y)
# initializing(network, initial_x, initial_y)

# sorted_index = selecting(network, x_train_scaled, y_train_scaled)
# network.setData(x_train_scaled[sorted_index[:20]],y_train_scaled[sorted_index[:20]])


    ## Set the random seed
    tf.random.set_seed(5)

    ## Find unsatisfied situation
    yo, loss, tape = network.forward()
    forward_info = [yo, loss, tape]
    
    undesired_index = tf.where(tf.math.abs(yo-network.y) > network.threshold_for_error)

    
#     print(tf.math.abs(yo-network.y))
#     print(network.threshold_for_error)
    print(undesired_index)
#     print(undesired_index.shape[0])

    ## Unsatisfied situation
    for i in range(undesired_index.shape[0]):

        k_data_num = undesired_index[i][0]
        k_l = undesired_index[i][1]

        undesired_data = tf.reshape(network.x[k_data_num,:], [1,-1])

        ## Remove the only data that does not meet the error term
        left_data = network.x[:k_data_num,:]
        right_data = network.x[k_data_num+1:,:]
        remain_tensor = tf.concat([left_data, right_data], 0)

    #     print(tf.subtract(remain_tensor, undesired_data).shape)

        while True:

            ## Find m-vector gamma: r
            gamma = tf.random.uniform(shape=[1,network.x.shape[1]])

            subtract_undesired_data = tf.subtract(remain_tensor, undesired_data)
            matmul_value = tf.matmul(gamma,tf.transpose(subtract_undesired_data))


            if tf.reduce_all(matmul_value != 0):

                while True:

                    ## Find the tiny value: zeta
                    zeta = tf.random.uniform(shape=[1])

                    if tf.reduce_all(tf.multiply(tf.add(zeta,matmul_value),tf.subtract(zeta,matmul_value))<0):
                        break

                break


        ## The weight of input layer to hidden layer I
        w10 = gamma
        w11 = gamma
        w12 = gamma

        W1_new = tf.transpose(tf.concat([w10,w11,w12],0))

    #     ## The bias of input layer to hidden layer I
        matual_value = tf.matmul(gamma,tf.transpose(undesired_data))

        b10 = tf.subtract(zeta,matual_value)
        b11 = -1*matual_value
        b12 = tf.subtract(-1*zeta,matual_value)
        b1_new = tf.reshape(tf.concat([b10,b11,b12],0),[3])

        ## The weight of hidden layer I to output layer
        gap = network.y[k_data_num, k_l]-yo[k_data_num, k_l]

        wo0_value = gap/zeta
        wo1_value = (-2*gap)/zeta
        wo2_value = gap/zeta

        wo0 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo0_value, shape=(1,-1))
        wo1 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo1_value, shape=(1,-1))
        wo2 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo2_value, shape=(1,-1))

        Wo_new = tf.concat([wo0,wo1,wo2],0)


    #     ## Add new neuroes to the network
        network.W1 = tf.Variable(tf.concat([network.W1, W1_new],1), tf.float32)
        network.b1 = tf.Variable(tf.concat([network.b1, b1_new],0), tf.float32)
        network.Wo = tf.Variable(tf.concat([network.Wo, Wo_new],0), tf.float32)

        yo, loss, tape = network.forward()
    #     print(tf.math.abs(yo[k_data_num,k_l]-network.y[k_data_num,k_l]) <= network.threshold_for_error)
        if tf.reduce_all(tf.math.abs(yo[k_data_num,k_l]-network.y[k_data_num,k_l]) <= network.threshold_for_error):
            
            if i==(undesired_index.shape[0]-1):
                network.acceptable = True
            
            print("Cramming success!")
            

        else:
            print("Cramming failed!")
        
# print(network.W1.shape[1])

### Construct a instance of network
- trained through the matching module, reorganizing module, and cramming module

In [ ]:
# def main():
    
# x_train_scaled, y_train_scaled = get_data(4)

x_train_data, x_test_data, y_train_data, y_test_data = get_data(4)

# x_train_scaled = x_train_scaled[:20]
# y_train_scaled = y_train_scaled[:20]

# x_train_scaled = sc.fit_transform(x_data[:100])
# x_test_scaled = sc.transform(x_data[100:])
# y_train_scaled = sc.fit_transform(y_data[:100])


x_train_scaled = sc.fit_transform(x_train_data)
x_test_scaled = sc.transform(x_test_data)
y_train_scaled = sc.fit_transform(y_train_data)

initial_n = x_train_scaled.shape[1]+2
number_data = x_train_scaled.shape[0]{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Import related package"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "INFO:tensorflow:Enabling eager execution\n",
      "INFO:tensorflow:Enabling v2 tensorshape\n",
      "INFO:tensorflow:Enabling resource variables\n",
      "INFO:tensorflow:Enabling tensor equality\n",
      "INFO:tensorflow:Enabling control flow v2\n"
     ]
    }
   ],
   "source": [
    "## Import tensorflow package for modeling\n",
    "import tensorflow as tf\n",
    "\n",
    "## Data processing\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "\n",
    "## Min-max normalization\n",
    "from sklearn.preprocessing import MinMaxScaler\n",
    "\n",
    "## Plot the graph\n",
    "import matplotlib.pyplot as plt\n",
    "%matplotlib inline\n",
    "\n",
    "## Initializing module\n",
    "from sklearn.linear_model import LinearRegression\n",
    "np.set_printoptions(suppress=True)\n",
    "\n",
    "## Copy module\n",
    "import copy"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Control memory usage space for GPU"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.3)\n",
    "sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))\n",
    "tf.compat.v1.keras.backend.set_session(sess)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Preprocessing the data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "def read(path):\n",
    "    return pd.read_csv(path)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "def buildTrain(train, pastWeek=4, futureWeek=4, defaultWeek=1):\n",
    "    X_train, Y_train = [], []\n",
    "    for i in range(train.shape[0]-futureWeek-pastWeek):\n",
    "        X = np.array(train.iloc[i:i+defaultWeek])\n",
    "        X = np.append(X,train[\"CCSP\"].iloc[i+defaultWeek:i+pastWeek])\n",
    "        X_train.append(X.reshape(X.size))\n",
    "        Y_train.append(np.array(train.iloc[i+pastWeek:i+pastWeek+futureWeek][\"CCSP\"]))\n",
    "    return np.array(X_train), np.array(Y_train)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Min-max normalization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "## Use min-max normalization to scale the data to the range from 1 to 0\n",
    "sc = MinMaxScaler(feature_range = (0, 1))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Design get_data() to get data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_data(futureWeek):\n",
    "    \n",
    "    ## Read weekly copper price data\n",
    "    path = \"WeeklyFinalData.csv\"\n",
    "    data = read(path)\n",
    "    \n",
    "    date = data[\"Date\"]\n",
    "    data.drop(\"Date\", axis=1, inplace=True)\n",
    "    \n",
    "    ## Add time lag (pastWeek=4, futureWeek=1)\n",
    "    x_data, y_data = buildTrain(data, futureWeek=futureWeek)\n",
    "    \n",
    "    ## Split the data to training data and test data\n",
    "    x_train_data = x_data[:int(x_data.shape[0]*0.8)]\n",
    "    x_test_data = x_data[int(x_data.shape[0]*0.8):]\n",
    "    y_train_data = y_data[:int(x_data.shape[0]*0.8)]\n",
    "    y_test_data = y_data[int(x_data.shape[0]*0.8):]\n",
    "\n",
    "\n",
    "    return (x_train_data, x_test_data, y_train_data, y_test_data)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Network class"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "class Network():\n",
    "    \n",
    "    def __init__(self, nb_neuro, x_train_scaled, y_train_scaled):\n",
    "        \n",
    "        # Stop criteria - threshold\n",
    "        self.threshold_for_error = 0.07\n",
    "        self.threshold_for_lr = 1e-6\n",
    "        \n",
    "        # Input data \n",
    "        self.x = tf.convert_to_tensor(x_train_scaled, np.float32)\n",
    "        self.y = tf.convert_to_tensor(y_train_scaled, np.float32)\n",
    "        \n",
    "        # Learning rate\n",
    "        self.learning_rate = 1e-2\n",
    "        \n",
    "        \n",
    "        # Hidden layer I\n",
    "        self.n_neurons_in_h1 = nb_neuro\n",
    "        self.W1 = tf.Variable(tf.random.truncated_normal([self.x.shape[1], self.n_neurons_in_h1], mean=0, stddev=1))\n",
    "        self.b1 = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1], mean=0, stddev=1))\n",
    "\n",
    "        # Output layer\n",
    "        self.Wo = tf.Variable(tf.random.truncated_normal([self.n_neurons_in_h1, self.y.shape[1]], mean=0, stddev=1))\n",
    "        self.bo = tf.Variable(tf.random.truncated_normal([self.y.shape[1]], mean=0, stddev=1))\n",
    "\n",
    "        # Whether the network is acceptable, default as False\n",
    "        self.acceptable = False\n",
    "    \n",
    "    ## Forecast the test data\n",
    "    def forecast(self, x_test_scaled):\n",
    "    \n",
    "        x_test_scaled = tf.cast(x_test_scaled, tf.float32)\n",
    "        activation_value = tf.nn.relu((tf.matmul(x_test_scaled, self.W1)+self.b1))\n",
    "        forecast_value = tf.matmul(activation_value,self.Wo)+self.bo\n",
    "       \n",
    "        return forecast_value\n",
    "\n",
    "    ## Reset the x and y data\n",
    "    def setData(self, x_train_scaled, y_train_scaled):\n",
    "        self.x = tf.convert_to_tensor(x_train_scaled, np.float32)\n",
    "        self.y = tf.convert_to_tensor(y_train_scaled, np.float32)\n",
    "    \n",
    "    ## Add the new data to the x and y data\n",
    "    def addData(self, new_x_train, new_y_train):\n",
    "\n",
    "        self.x = tf.concat([self.x, new_x_train.reshape(1,-1)],0)\n",
    "        self.y = tf.concat([self.y, new_y_train.reshape(1,-1)],0)\n",
    "    \n",
    "    ## forward operation\n",
    "    def forward(self,  reg_strength= 0):\n",
    "        with tf.GradientTape() as tape:\n",
    "\n",
    "            y1 = tf.nn.relu((tf.matmul(self.x, self.W1)+self.b1))\n",
    "            yo = (tf.matmul(y1,self.Wo)+self.bo)\n",
    "\n",
    "            # performance measure\n",
    "            diff = yo-self.y\n",
    "            loss = tf.reduce_mean(diff**2) + (reg_strength/(self.Wo.shape[1]*(self.Wo.shape[0]+1)+self.W1.shape[1]*(self.W1.shape[0]+1))) * ((tf.nn.l2_loss(self.W1) + tf.nn.l2_loss(self.Wo) + tf.nn.l2_loss(self.b1) + tf.nn.l2_loss(self.bo))*2)\n",
    "\n",
    "        return(yo, loss, tape)\n",
    "\n",
    "    # backward operation\n",
    "    def backward_Adam(self,tape,loss):\n",
    "\n",
    "        optimizer = tf.optimizers.Adam(self.learning_rate)\n",
    "        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])\n",
    "        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))\n",
    "    \n",
    "    def backward_RMS(self,tape,loss):\n",
    "\n",
    "        optimizer = tf.keras.optimizers.RMSprop(self.learning_rate)\n",
    "        gradients = tape.gradient(loss, [self.W1, self.Wo, self.b1, self.bo])\n",
    "        optimizer.apply_gradients(zip(gradients, [self.W1, self.Wo, self.b1, self.bo]))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Initializing module (Check)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "def initializing(network, initial_x, initial_y):\n",
    "    \n",
    "    ## Find each minimum output value y\n",
    "    min_y = tf.reduce_min(initial_y, axis=0)\n",
    "    \n",
    "    ## Subtract min_y from each y\n",
    "    res_y = initial_y-min_y\n",
    "    \n",
    "    ## Use linear regression to find the initial W1,b1,Wo,bo\n",
    "    reg = LinearRegression().fit(initial_x, res_y)\n",
    "\n",
    "    ## Set up the initial parameter of the network\n",
    "    network.W1 = tf.Variable(tf.cast(tf.transpose(reg.coef_), tf.float32))\n",
    "    network.b1 = tf.Variable(tf.convert_to_tensor(reg.intercept_, tf.float32))\n",
    "    network.Wo = tf.Variable(tf.convert_to_tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], tf.float32))\n",
    "    network.bo = tf.Variable(tf.cast(min_y, tf.float32))\n",
    "\n",
    "    ## Set up the acceptable of the initial network as True\n",
    "    network.acceptable =True"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Selecting module (Check)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "def selecting(network, x_train_scaled, y_train_scaled):\n",
    "    \n",
    "    \n",
    "    loss = []\n",
    "    temp_network = copy.deepcopy(network)\n",
    "    \n",
    "    ## Put each data into network to calculate the loss value\n",
    "    for i in range(x_train_scaled.shape[0]):\n",
    "        temp_network.setData(x_train_scaled[i].reshape(1,-1), y_train_scaled[i].reshape(1,-1))\n",
    "        loss.append((temp_network.forward()[1].numpy(),i))\n",
    "\n",
    "    ## Sort the data according to the loss value from smallest to largest, and save the data index in sorted_index\n",
    "    sorted_index = [sorted_data[1] for sorted_data in sorted(loss, key = lambda x:x[0])]\n",
    "\n",
    "    ## Print out some info for debug\n",
    "    print(\"First:\",loss[sorted_index[0]])\n",
    "    print(\"Selecting module finish!\")\n",
    "    \n",
    "    return sorted_index"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Matching module (Check)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [],
   "source": [
    "def matching(network):\n",
    "\n",
    "    ## Set up the learning rate of the network\n",
    "    network.learning_rate = 1e-3\n",
    "    \n",
    "\n",
    "    while True:\n",
    "        \n",
    "        ## Get the loss value of the current network architecture\n",
    "        yo, loss, tape = network.forward()\n",
    "\n",
    "        ## Identify that all forecast value has met the error term\n",
    "        if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):\n",
    "            \n",
    "            ## If true, set the acceptable of the network as true and return it\n",
    "            network.acceptable = True\n",
    "            print(\"Matching finished - the network is acceptable\")\n",
    "            return(network)\n",
    "\n",
    "\n",
    "        ## If the error is not satisfied, continue to tunning the learning rate of the network\n",
    "        else:\n",
    "            \n",
    "            # Save the current papameter\n",
    "            network_pre = copy.deepcopy(network)\n",
    "            \n",
    "            # Stroe the last loss value\n",
    "            loss_pre = loss\n",
    "            \n",
    "            # Backward and check the loss performance of the network with new learning rate\n",
    "            network.backward_Adam(tape,loss)\n",
    "            yo, loss, tape = network.forward()\n",
    "\n",
    "            # Confirm whether the loss value of the adjusted network is smaller than the current one\n",
    "            if loss < loss_pre:\n",
    "\n",
    "                # If true, multiply the learning rate by 1.2\n",
    "                network.learning_rate *= 1.2\n",
    "\n",
    "            # On the contrary, reduce the learning rate\n",
    "            else:         \n",
    "                \n",
    "                # Identify whether the current learning rate is less than the threshold\n",
    "                if network.learning_rate <= network.threshold_for_lr:\n",
    "                    \n",
    "                    # If true, set the acceptable of the network as false and return it\n",
    "                    network.acceptable = False\n",
    "                    print(\"Matching finished - the network is Unacceptable\")\n",
    "                    return(network)\n",
    "\n",
    "                # On the contrary, restore w and adjust the learning rate\n",
    "                else:\n",
    "                    \n",
    "                    # Restore the papameter of the network\n",
    "                    network = network_pre\n",
    "                    network.learning_rate *= 0.7"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Cramming module (Check)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "def cramming(network):\n",
    "    \n",
    "\n",
    "    ## Set the random seed\n",
    "    tf.random.set_seed(5)\n",
    "\n",
    "    ## Find unsatisfied data:K\n",
    "    yo, loss, tape = network.forward()\n",
    "    undesired_index = tf.where(tf.math.abs(yo-network.y) > network.threshold_for_error)\n",
    "\n",
    "    ## Print out the undesired_index for debug\n",
    "    print(undesired_index)\n",
    "\n",
    "\n",
    "    ## Unsatisfied situation\n",
    "    for i in range(undesired_index.shape[0]):\n",
    "\n",
    "        ## Find the index of the unsatisfied data\n",
    "        k_data_num = undesired_index[i][0]\n",
    "        k_l = undesired_index[i][1]\n",
    "\n",
    "        undesired_data = tf.reshape(network.x[k_data_num,:], [1,-1])\n",
    "\n",
    "        ## Remove the data that does not meet the error term\n",
    "        left_data = network.x[:k_data_num,:]\n",
    "        right_data = network.x[k_data_num+1:,:]\n",
    "        remain_tensor = tf.concat([left_data, right_data], 0)\n",
    "\n",
    "        \n",
    "        ## Use the random method to find out the gamma and zeta\n",
    "        while True:\n",
    "\n",
    "            ## Find m-vector gamma: r\n",
    "            ## Use the random method to generate the gamma that can make the conditions met\n",
    "            gamma = tf.random.uniform(shape=[1,network.x.shape[1]])\n",
    "            subtract_undesired_data = tf.subtract(remain_tensor, undesired_data)\n",
    "            matmul_value = tf.matmul(gamma,tf.transpose(subtract_undesired_data))\n",
    "\n",
    "\n",
    "            if tf.reduce_all(matmul_value != 0):\n",
    "\n",
    "                while True:\n",
    "\n",
    "                    ## Find the tiny value: zeta\n",
    "                    ## Use the random method to generate the zeta that can make the conditions met\n",
    "                    zeta = tf.random.uniform(shape=[1])\n",
    "\n",
    "                    if tf.reduce_all(tf.multiply(tf.add(zeta,matmul_value),tf.subtract(zeta,matmul_value))<0):\n",
    "                        break\n",
    "\n",
    "                break\n",
    "\n",
    "\n",
    "        ## The weight of input layer to hidden layer I\n",
    "        w10 = gamma\n",
    "        w11 = gamma\n",
    "        w12 = gamma\n",
    "        \n",
    "        W1_new = tf.transpose(tf.concat([w10,w11,w12],0))\n",
    "\n",
    "        ## The bias of input layer to hidden layer I\n",
    "        matual_value = tf.matmul(gamma,tf.transpose(undesired_data))\n",
    "\n",
    "        b10 = tf.subtract(zeta,matual_value)\n",
    "        b11 = -1*matual_value\n",
    "        b12 = tf.subtract(-1*zeta,matual_value)\n",
    "        b1_new = tf.reshape(tf.concat([b10,b11,b12],0),[3])\n",
    "\n",
    "        ## The weight of hidden layer I to output layer\n",
    "        gap = network.y[k_data_num, k_l]-yo[k_data_num, k_l]\n",
    "\n",
    "        wo0_value = gap/zeta\n",
    "        wo1_value = (-2*gap)/zeta\n",
    "        wo2_value = gap/zeta\n",
    "\n",
    "        wo0 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo0_value, shape=(1,-1))\n",
    "        wo1 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo1_value, shape=(1,-1))\n",
    "        wo2 = tf.reshape(tf.one_hot(k_l,4,dtype=tf.float32) * wo2_value, shape=(1,-1))\n",
    "\n",
    "        Wo_new = tf.concat([wo0,wo1,wo2],0)\n",
    "\n",
    "        ## Add new neuroes to the network\n",
    "        network.W1 = tf.Variable(tf.concat([network.W1, W1_new],1), tf.float32)\n",
    "        network.b1 = tf.Variable(tf.concat([network.b1, b1_new],0), tf.float32)\n",
    "        network.Wo = tf.Variable(tf.concat([network.Wo, Wo_new],0), tf.float32)\n",
    "\n",
    "        yo, loss, tape = network.forward()\n",
    "   \n",
    "        ## Determine if cramming is successful and print out the corresponding information\n",
    "        if tf.reduce_all(tf.math.abs(yo[k_data_num,k_l]-network.y[k_data_num,k_l]) <= network.threshold_for_error):\n",
    "            \n",
    "            ## If the cramming process is complete, set the acceptable of the network as true\n",
    "            if i==(undesired_index.shape[0]-1):\n",
    "                network.acceptable = True\n",
    "            \n",
    "            print(\"Cramming success!\")\n",
    "            \n",
    "        \n",
    "        else:\n",
    "            print(\"Cramming failed!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Regularizing module (Check)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "def regularizing(network):\n",
    "\n",
    "    ## Set up the learning rate of the network\n",
    "    network.learning_rate = 1e-3\n",
    "\n",
    "    ## Set epoch to 100\n",
    "    for i in range(100):\n",
    "\n",
    "        ## Store the parameter of the network\n",
    "        network_pre = copy.deepcopy(network)\n",
    "        yo, loss, tape = network.forward(1e-2)\n",
    "        loss_pre = loss\n",
    "\n",
    "        ## Backward operation to obtain w'\n",
    "        network.backward_Adam(tape, loss)\n",
    "        yo, loss, tape = network.forward(1e-2)\n",
    "\n",
    "         # Confirm whether the adjusted loss value is smaller than the current one\n",
    "        if loss <= loss_pre:\n",
    "            \n",
    "            ## Identify that all forecast value has met the error term\n",
    "            if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):\n",
    "                \n",
    "                ## If true, multiply the learning rate by 1.2\n",
    "                network.learning_rate *= 1.2\n",
    "                print(\"Regularizing process\")\n",
    "\n",
    "            else:\n",
    "\n",
    "                ## Else, restore w and end the process\n",
    "                network = network_pre\n",
    "                print(\"Regularizing finished(A)\")\n",
    "                return(network)\n",
    "\n",
    "        # If the adjusted loss value is not smaller than the current one\n",
    "        else:\n",
    "           \n",
    "\n",
    "            ## If the learning rate is greater than the threshold for learning rate\n",
    "            if network.learning_rate > network.threshold_for_lr:\n",
    "                \n",
    "                ## Restore the w and multiply the learning rate by 0.7\n",
    "                network = network_pre\n",
    "                network.learning_rate *= 0.7\n",
    "\n",
    "             ## If the learning rate is smaller than the threshold for learning rate\n",
    "            else:\n",
    "                \n",
    "                ## Restore the w\n",
    "                network = network_pre\n",
    "                print(\"Regularizing finished(B)\")\n",
    "                return(network)\n",
    "\n",
    "        if i == 99:\n",
    "            \n",
    "            return(network)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Reorganizing module (Check)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [],
   "source": [
    "def reorganizing(network):\n",
    "    \n",
    "    ## Set up the k = 1, and p = the number of hidden node\n",
    "    k = 1\n",
    "    p = network.W1.shape[1]\n",
    "\n",
    "    while True:\n",
    "\n",
    "        ## If k > p, end of Process\n",
    "        if k > p:\n",
    "\n",
    "            print(\"The number of neuro: \",p)\n",
    "            return(network)\n",
    "\n",
    "        ## Else, Process is ongoing\n",
    "        else:\n",
    "\n",
    "            ## Using the regularizing module to adjust the network\n",
    "            network = regularizing(network)\n",
    "            \n",
    "            ## Store the network and w\n",
    "            network_pre = copy.deepcopy(network)\n",
    "\n",
    "            ## Set up the acceptable of the network as false\n",
    "            network.acceptable = False\n",
    "\n",
    "            ## Ignore the K hidden node\n",
    "            network.W1 = tf.Variable(tf.concat([network.W1[:,:k-1],network.W1[:,k:]],1))\n",
    "            network.b1 = tf.Variable(tf.concat([network.b1[:k-1],network.b1[k:]],0))\n",
    "            network.Wo = tf.Variable(tf.concat([network.Wo[:k-1,:],network.Wo[k:,:]],0))\n",
    "\n",
    "            ## Using the matching module to adjust the network\n",
    "            network = matching(network)\n",
    "\n",
    "            ## If the resulting network is acceptable, this means that the k hidden node can be removed\n",
    "            if network.acceptable:\n",
    "\n",
    "                print(\"Drop out the nero number: %d / %d\" %(k, p))\n",
    "                \n",
    "                ## p--\n",
    "                p-=1\n",
    "\n",
    "            ## Else, it means that the k hidden node cannot be removed\n",
    "            else:\n",
    "                \n",
    "                ## Restore the network and w\n",
    "                network = network_pre\n",
    "                print(\"Cannot drop out the nero number: %d / %d\" %(k, p))\n",
    "                \n",
    "                ## k++\n",
    "                k+=1\n",
    "                    "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Construct a instance of network\n",
    "- trained through the matching module, reorganizing module, and cramming module"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "The data number: 0\n",
      "First: (0.04087187, 0)\n",
      "Selecting module finish!\n",
      "tf.Tensor(\n",
      "[[0.00000376 0.00000101 0.00000083 0.        ]\n",
      " [0.00000203 0.00000203 0.00000036 0.00000018]\n",
      " [0.00000662 0.00000179 0.00000006 0.00000006]\n",
      " [0.00000101 0.00000089 0.00000006 0.00000024]\n",
      " [0.00000167 0.00000328 0.00000036 0.00000018]\n",
      " [0.00000322 0.00000274 0.00000089 0.00000077]\n",
      " [0.00000542 0.00000399 0.0000003  0.00000012]\n",
      " [0.00000131 0.0000034  0.         0.00000054]\n",
      " [0.00000119 0.00000644 0.0000003  0.00000042]\n",
      " [0.00000149 0.00000054 0.0000003  0.00000066]\n",
      " [0.00000703 0.00000638 0.0000003  0.00000048]\n",
      " [0.00000715 0.00000376 0.00000012 0.00000006]\n",
      " [0.00000644 0.00000536 0.00000006 0.00000024]\n",
      " [0.00000423 0.0000059  0.00000012 0.        ]\n",
      " [0.00000286 0.00000012 0.00000012 0.        ]\n",
      " [0.00000066 0.00000417 0.00000012 0.        ]\n",
      " [0.00000149 0.00000942 0.00000036 0.        ]\n",
      " [0.00000471 0.00000727 0.00000012 0.00000024]\n",
      " [0.00000232 0.00000727 0.00000012 0.00000042]\n",
      " [0.3753181  0.08476233 0.1098395  0.05810875]], shape=(20, 4), dtype=float32)\n",
      "Matching finished - the network is acceptable\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 1 / 4\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is acceptable\n",
      "Drop out the nero number: 2 / 4\n",
      "Regularizing process\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 2 / 3\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is acceptable\n",
      "Drop out the nero number: 3 / 3\n",
      "The number of neuro:  2\n",
      "True\n",
      "The data number: 1\n",
      "First: (0.005836749, 2)\n",
      "Selecting module finish!\n",
      "tf.Tensor(\n",
      "[[0.01986951 0.06945199 0.00541383 0.06999594]\n",
      " [0.05871511 0.00397789 0.02099007 0.04159617]\n",
      " [0.004067   0.02234453 0.00597948 0.01225376]\n",
      " [0.01995784 0.01963151 0.00542492 0.02590179]\n",
      " [0.01618862 0.0057506  0.0060513  0.05726844]\n",
      " [0.03818065 0.05319911 0.02343881 0.0609169 ]\n",
      " [0.05340415 0.00608885 0.03061056 0.01753896]\n",
      " [0.00320423 0.02202916 0.00245231 0.02899194]\n",
      " [0.02015615 0.01574165 0.00377452 0.01571155]\n",
      " [0.01305854 0.01949537 0.01101488 0.00405598]\n",
      " [0.02320278 0.00988626 0.01537085 0.01226848]\n",
      " [0.00219345 0.01390648 0.00108773 0.0034852 ]\n",
      " [0.00594234 0.00994688 0.00383151 0.02411383]\n",
      " [0.00248748 0.01619649 0.00241369 0.0020079 ]\n",
      " [0.01005888 0.01197624 0.00037289 0.00779653]\n",
      " [0.01191753 0.00127125 0.00007862 0.00293905]\n",
      " [0.00865227 0.01206332 0.00301963 0.01458722]\n",
      " [0.01752526 0.01526564 0.00288206 0.05612427]\n",
      " [0.03683561 0.02470821 0.02414542 0.00713044]\n",
      " [0.06169957 0.0363251  0.02614421 0.04571098]\n",
      " [0.12292713 0.04553246 0.0421831  0.06620634]], shape=(21, 4), dtype=float32)\n",
      "Matching finished - the network is acceptable\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 1 / 2\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 2 / 2\n",
      "The number of neuro:  2\n",
      "True\n",
      "The data number: 2\n",
      "First: (0.0073913517, 3)\n",
      "Selecting module finish!\n",
      "tf.Tensor(\n",
      "[[0.01891488 0.06943083 0.00374341 0.06346637]\n",
      " [0.0591411  0.00414187 0.02083534 0.03503406]\n",
      " [0.00053662 0.02310884 0.00169122 0.00201178]\n",
      " [0.03059536 0.02213973 0.01339716 0.01348954]\n",
      " [0.02584362 0.00358415 0.01352942 0.06917381]\n",
      " [0.04843414 0.05615073 0.02988821 0.06860632]\n",
      " [0.06802815 0.01087093 0.03834462 0.02401292]\n",
      " [0.01448911 0.01749891 0.0067783  0.03158879]\n",
      " [0.01006329 0.02034527 0.00090384 0.01591206]\n",
      " [0.00594378 0.01611853 0.00906396 0.00427514]\n",
      " [0.01907438 0.01223135 0.0147388  0.0107519 ]\n",
      " [0.00552613 0.01538944 0.00036722 0.00450093]\n",
      " [0.00639164 0.00970536 0.00317597 0.0229519 ]\n",
      " [0.00302768 0.01681149 0.00183845 0.0006426 ]\n",
      " [0.01049805 0.01157576 0.00037295 0.00571132]\n",
      " [0.010575   0.00038099 0.00096673 0.00493491]\n",
      " [0.00798494 0.01175755 0.00254637 0.01545465]\n",
      " [0.01755375 0.01559281 0.00284499 0.05569738]\n",
      " [0.03546631 0.0249052  0.02279931 0.0038932 ]\n",
      " [0.04081815 0.02747416 0.01834261 0.04259914]\n",
      " [0.06994873 0.01900262 0.04616344 0.06317395]\n",
      " [0.02487671 0.05961794 0.0739131  0.14116943]], shape=(22, 4), dtype=float32)\n",
      "Matching finished - the network is Unacceptable\n",
      "tf.Tensor(\n",
      "[[21  2]\n",
      " [21  3]], shape=(2, 2), dtype=int64)\n",
      "Cramming success!\n",
      "Cramming success!\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is acceptable\n",
      "Drop out the nero number: 1 / 8\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 1 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 2 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 3 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 4 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 5 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 6 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 7 / 7\n",
      "The number of neuro:  7\n",
      "True\n",
      "The data number: 3\n",
      "First: (0.0044059106, 0)\n",
      "Selecting module finish!\n",
      "tf.Tensor(\n",
      "[[0.00270122 0.05031335 0.004017   0.03844976]\n",
      " [0.04448819 0.01599878 0.02649993 0.02227044]\n",
      " [0.02429503 0.04071397 0.00990123 0.00501537]\n",
      " [0.05043101 0.03115338 0.00480318 0.0369904 ]\n",
      " [0.04402614 0.0030123  0.00055963 0.03940773]\n",
      " [0.04567969 0.04428834 0.04054236 0.05292243]\n",
      " [0.06202668 0.00450814 0.0301522  0.00115579]\n",
      " [0.01578808 0.05019665 0.01078683 0.02014494]\n",
      " [0.05551755 0.02411366 0.01563257 0.01429611]\n",
      " [0.04698819 0.05466449 0.02104568 0.00031328]\n",
      " [0.04341674 0.00833821 0.00571752 0.00607431]\n",
      " [0.01464945 0.02474916 0.00680107 0.01255655]\n",
      " [0.03588998 0.01932651 0.02388215 0.0053798 ]\n",
      " [0.00782007 0.02378225 0.00433666 0.00110096]\n",
      " [0.01105583 0.00845152 0.00185281 0.01413745]\n",
      " [0.03538662 0.02096719 0.02153969 0.02620167]\n",
      " [0.00590092 0.00852877 0.01037019 0.01278502]\n",
      " [0.00064981 0.00039035 0.00117934 0.03708601]\n",
      " [0.02863091 0.0195784  0.01249391 0.0119521 ]\n",
      " [0.03817427 0.01020211 0.06115746 0.0664373 ]\n",
      " [0.02512991 0.00173354 0.04563153 0.06992722]\n",
      " [0.04077661 0.05357724 0.00101405 0.00196761]\n",
      " [0.00327998 0.05367261 0.10359865 0.06324512]], shape=(23, 4), dtype=float32)\n",
      "Matching finished - the network is acceptable\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 1 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 2 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 3 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 4 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 5 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 6 / 7\n",
      "Regularizing finished(A)\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 7 / 7\n",
      "The number of neuro:  7\n",
      "True\n",
      "The data number: 4\n",
      "First: (0.002363425, 1)\n",
      "Selecting module finish!\n",
      "tf.Tensor(\n",
      "[[0.00280309 0.04459119 0.00416076 0.02924377]\n",
      " [0.03908265 0.02864522 0.03083056 0.02265316]\n",
      " [0.02939755 0.05286473 0.01738513 0.0113861 ]\n",
      " [0.03612083 0.02249306 0.00309283 0.03656989]\n",
      " [0.0300414  0.00765967 0.00123298 0.03588206]\n",
      " [0.03656173 0.03746837 0.04481649 0.05275112]\n",
      " [0.06178403 0.00103337 0.01903224 0.00094146]\n",
      " [0.01752383 0.04755121 0.00236714 0.01868159]\n",
      " [0.05563414 0.01991326 0.00200176 0.01384783]\n",
      " [0.04874766 0.05244195 0.01372302 0.0022676 ]\n",
      " [0.04556781 0.0071913  0.01073807 0.00484043]\n",
      " [0.0166347  0.02791047 0.00483263 0.01246625]\n",
      " [0.03749239 0.02075368 0.01721376 0.00139242]\n",
      " [0.00735921 0.02219677 0.00275809 0.0046615 ]\n",
      " [0.01134205 0.01055366 0.00344992 0.0121277 ]\n",
      " [0.03702289 0.02450341 0.01676708 0.03082329]\n",
      " [0.00571764 0.01155883 0.00795376 0.01694244]\n",
      " [0.00197035 0.00175089 0.0002107  0.03686905]\n",
      " [0.02902073 0.02342266 0.02062023 0.00960314]\n",
      " [0.01818973 0.01386762 0.0180735  0.03414905]\n",
      " [0.02675033 0.00392538 0.05854267 0.06999993]\n",
      " [0.03643638 0.04682857 0.00265276 0.00093615]\n",
      " [0.01505691 0.03119022 0.05767977 0.03376508]\n",
      " [0.08063775 0.03520143 0.04030478 0.00936145]], shape=(24, 4), dtype=float32)\n",
      "Matching finished - the network is acceptable\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 1 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 2 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 3 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 4 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 5 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 6 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 7 / 7\n",
      "The number of neuro:  7\n",
      "True\n",
      "The data number: 5\n",
      "First: (0.0016563192, 0)\n",
      "Selecting module finish!\n",
      "tf.Tensor(\n",
      "[[0.00307107 0.04423708 0.00796187 0.02435124]\n",
      " [0.03755474 0.03015769 0.03021204 0.02019268]\n",
      " [0.03201985 0.05535841 0.02065748 0.012564  ]\n",
      " [0.03047305 0.01617455 0.00087762 0.03816944]\n",
      " [0.02524745 0.01450527 0.00140059 0.03246343]\n",
      " [0.03610158 0.03484827 0.04266661 0.05093956]\n",
      " [0.06801021 0.0047794  0.01593238 0.00074631]\n",
      " [0.01268172 0.04293692 0.00574893 0.01606143]\n",
      " [0.04964817 0.01414698 0.00119019 0.01451397]\n",
      " [0.04387939 0.04859293 0.01140189 0.00418478]\n",
      " [0.04334843 0.00617379 0.01118875 0.00478023]\n",
      " [0.01935565 0.02892017 0.00004345 0.01016891]\n",
      " [0.03762221 0.0187574  0.01049048 0.00315005]\n",
      " [0.00727743 0.0194205  0.00061375 0.00709546]\n",
      " [0.01204914 0.01303524 0.00410312 0.00955564]\n",
      " [0.0357132  0.02642453 0.0162344  0.03391105]\n",
      " [0.00488478 0.01425177 0.00993854 0.02049941]\n",
      " [0.00360602 0.00411475 0.00417089 0.03350145]\n",
      " [0.02780861 0.02598739 0.01882666 0.013511  ]\n",
      " [0.00673497 0.02512771 0.00255746 0.02404618]\n",
      " [0.02115464 0.00279635 0.05968755 0.06269383]\n",
      " [0.04179537 0.04838204 0.00541317 0.00336951]\n",
      " [0.0269106  0.01965541 0.04145443 0.02673441]\n",
      " [0.06623572 0.021568   0.02120686 0.00012016]\n",
      " [0.02031642 0.06292939 0.04379708 0.01827955]], shape=(25, 4), dtype=float32)\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 1 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 2 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 3 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 4 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 5 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 6 / 7\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Regularizing process\n",
      "Matching finished - the network is Unacceptable\n",
      "Cannot drop out the nero number: 7 / 7\n",
      "The number of neuro:  7\n",
      "True\n"
     ]
    }
   ],
   "source": [
    "## Call the help method \"get_data(...)\" to get the training data and test data \n",
    "x_train_data, x_test_data, y_train_data, y_test_data = get_data(4)\n",
    "\n",
    "## Use min-max normalization to normalize data in a range of 0 to 1\n",
    "x_train_scaled = sc.fit_transform(x_train_data)\n",
    "x_test_scaled = sc.transform(x_test_data)\n",
    "y_train_scaled = sc.fit_transform(y_train_data)\n",
    "\n",
    "## Use the first 25 training data to debug\n",
    "x_train_scaled = x_train_scaled[:25]\n",
    "y_train_scaled = y_train_scaled[:25]\n",
    "\n",
    "## Pick up m+1 data that are linearly independent as the initial m+1 training data \n",
    "initial_x = x_train_scaled[:x_train_scaled.shape[1]+1]\n",
    "initial_y = y_train_scaled[:x_train_scaled.shape[1]+1]\n",
    "\n",
    "## The remaining data (that exclude initial data)\n",
    "x_train_scaled = x_train_scaled[x_train_scaled.shape[1]+1:]\n",
    "y_train_scaled = y_train_scaled[x_train_scaled.shape[1]+1:]\n",
    "\n",
    "## Construct the network object with 4 neuros\n",
    "network = Network(4, initial_x, initial_y)\n",
    "\n",
    "## Use initilize module to set up the initial network\n",
    "initializing(network, initial_x, initial_y)\n",
    "\n",
    "\n",
    "## Training of all data\n",
    "for i in range(0, x_train_scaled.shape[0]):\n",
    "    \n",
    "    ## Print out some info for debug\n",
    "    print(\"The data number: %d\"%i)\n",
    "    \n",
    "    ## Get the data index by selecting module to sort the data by the loss value from smallest to largest\n",
    "    sorted_index = selecting(network, x_train_scaled, y_train_scaled)\n",
    "    \n",
    "    ## Add new data for training\n",
    "    network.addData(x_train_scaled[sorted_index[0]], y_train_scaled[sorted_index[0]])\n",
    "    x_train_scaled = np.delete(x_train_scaled, sorted_index[0], 0)\n",
    "    y_train_scaled = np.delete(y_train_scaled, sorted_index[0], 0)\n",
    "    \n",
    "    yo, loss, tape = network.forward()\n",
    "    \n",
    "    ## Print out some information for debug\n",
    "    print(tf.math.abs(yo-network.y))\n",
    "    \n",
    "    ## Determine whether the forecast value can meet the error term\n",
    "    if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):\n",
    "        \n",
    "        ## If true, set up the acceptable of the network as true\n",
    "        network.acceptable = True\n",
    "        \n",
    "        ## Use reorganizing module to adjust the model\n",
    "        network = reorganizing(network)\n",
    " \n",
    "    else:\n",
    "        \n",
    "        ## If true, set up the acceptable of the network as false\n",
    "        network.acceptable = False\n",
    "        network_pre = copy.deepcopy(network)\n",
    "        \n",
    "        ## Use matching module to adjust the model\n",
    "        network = matching(network)\n",
    "        \n",
    "        ## If the output of the matching module is an acceptable network, use the reorganization module to adjust the model\n",
    "        if network.acceptable:\n",
    "            network = reorganizing(network)\n",
    " \n",
    "        ## Else (if the output of the matching module is an unacceptable network)\n",
    "        else:\n",
    "            network = network_pre\n",
    "            \n",
    "            ## Use cramming module and reorganizing module to adjust the model\n",
    "            cramming(network)\n",
    "            network = reorganizing(network)\n",
    "\n",
    "    ## Print out the model status\n",
    "    print(network.acceptable)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.legend.Legend at 0x7fef3899c2b0>"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "yo, loss, tape = network.forward()\n",
    "plt.plot(sc.inverse_transform(yo)[:,0], label=\"LLAAT\")\n",
    "plt.plot(sc.inverse_transform(network.y)[:,0], label=\"Actual\")\n",
    "plt.legend()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.legend.Legend at 0x7fef38882b00>"
      ]
     },
     "execution_count": 16,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "forecast_value = network.forecast(x_test_scaled)\n",
    "\n",
    "plt.plot(sc.inverse_transform(forecast_value)[:,0], label=\"LLAAT\")\n",
    "plt.plot(y_test_data[:,0], label=\"Actual\")\n",
    "plt.legend()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

Date,CCSP,CDCS,ADCS,CDWS,GL,NDCS,NRBCON,SL,ZSCS,EURUSD,USDCLP,USDCNY,USDPEN,USInflation,ChinaInflation
2011/10/31,58960,7864,2126.4,423475,1737.46,18794.8,2.63,34.09,1941.7,1.37,494.29,6.35,2.71,3.42,4.46
2011/11/7,57595,7654.7,2116.47,411505,1779.54,18253,2.66,34.51,1932.15,1.37,499.35,6.35,2.7,3.4,4.2
2011/11/14,57775,7625.9,2108.35,401500,1753.61,17876.4,2.55,33.28,1931.6,1.35,508.3,6.35,2.7,3.4,4.2
2011/11/21,56015,7248.95,2017.1,396255,1689.11,17287.8,2.51,31.79,1910.55,1.34,521.86,6.36,2.7,3.4,4.2
2011/11/28,57365,7684.15,2073.6,388955,1733.17,17230.05,2.56,32.4,2018.05,1.34,518.34,6.35,2.7,3.24,4.16
2011/12/5,58135,7798.95,2086.2,388505,1722.21,18360.5,2.6,32.22,2014.1,1.34,510.21,6.34,2.7,3,4.1
2011/12/12,55970,7373.15,1986.45,382815,1608.02,18121.4,2.53,29.92,1880.95,1.3,517.12,6.34,2.7,3,4.1
2011/12/19,54960,7448.15,1981.45,372070,1607.29,18657.2,2.6,29.15,1840.8,1.3,519.82,6.33,2.69,3,4.1
2011/12/26,55205,7505.75,1987.8,371370,1572.42,18228.2,2.67,28.06,1814.1,1.3,520.46,6.31,2.7,3,4.1
2012/1/2,55935,7617.33,2036.2,369697.5,1604.16,18793.4,2.74,28.93,1843.45,1.29,512.32,6.3,2.69,2.9,4.5
2012/1/9,56000,7794.9,2121.15,362320,1634.32,19439.8,2.75,29.76,1926.15,1.28,505.98,6.31,2.69,2.9,4.5
2012/1/16,58151.67,8210.75,2173.25,352090,1655.3,19767,2.79,30.66,1989.05,1.28,495.2,6.32,2.69,2.9,4.5
2012/1/23,59508.33,8431.3,2220.6,340215,1701.86,20977,2.84,32.93,2125.95,1.31,488.33,6.31,2.69,2.9,4.5
2012/1/30,59320,8398.45,2208.6,329605,1738.9,20982.2,2.89,33.64,2105.1,1.31,484.84,6.3,2.69,2.9,3.72
2012/2/6,59950,8536.45,2213.8,317170,1729.24,21348,2.97,33.8,2106.75,1.32,478.85,6.29,2.69,2.9,3.2
2012/2/13,59370,8320.65,2147.9,311570,1724.05,19971,3.01,33.47,1989.9,1.31,482.85,6.29,2.68,2.9,3.2
2012/2/20,59340,8410.95,2220.7,305080,1764.12,19968.6,3.09,34.54,2022.75,1.33,481.14,6.3,2.68,2.9,3.2
2012/2/27,59780,8570.75,2291.15,295400,1735.49,19543.4,3.21,35.42,2093.05,1.33,479.67,6.3,2.68,2.82,3.36
2012/3/5,59430,8382.75,2192.15,281270,1695.53,18872.25,3.28,33.68,2026.6,1.32,487.18,6.31,2.67,2.7,3.6
2012/3/12,59895,8519.65,2202.65,269825,1666.64,19230.9,3.34,32.8,2076.32,1.31,483.8,6.32,2.67,2.7,3.6
2012/3/19,59540,8456.1,2174.2,258470,1650.27,18626.8,3.36,32.18,2015.6,1.32,485.42,6.3,2.67,2.7,3.6
2012/3/26,59455,8472.5,2121.7,255925,1672.27,17620.2,3.39,32.37,2003.7,1.33,486.8,6.29,2.67,2.7,3.6
2012/4/2,59131,8465.6,2071.2,261726.67,1642.13,18182.8,3.36,32.07,1999.87,1.32,483.16,6.29,2.67,2.3,3.4
2012/4/9,58530,8122.6,2045.12,266643.33,1657.72,18238.9,3.31,31.7,2003.73,1.31,484.91,6.29,2.66,2.3,3.4
2012/4/16,57130,8137.1,2031.4,261555,1646.14,17602.6,3.2,31.59,1991.25,1.31,486.71,6.29,2.65,2.3,3.4
2012/4/23,57772,8340.8,2040.7,255860,1647.83,17692.6,3.18,30.91,2002.85,1.32,485.98,6.29,2.65,2.3,3.4
2012/4/30,58348,8401.7,2059.2,237575,1651.81,17474.8,3.07,30.59,2016.35,1.32,483.37,6.28,2.64,1.82,3.08
2012/5/7,57818,8178.1,2011.12,224007.5,1601.17,17223.6,3,29.3,1961.52,1.3,485.79,6.29,2.65,1.7,3
2012/5/14,56221,7757.9,2000.45,217725,1561.44,16919,2.92,27.94,1909.25,1.27,499.74,6.32,2.67,1.7,3
2012/5/21,56257,7698.3,1983.4,224325,1570.51,16930.2,2.9,28.23,1890.3,1.26,507.81,6.32,2.68,1.7,3
2012/5/28,56019,7536.5,1964,227500,1575.09,16363.6,2.81,28.05,1890.15,1.24,515.5,6.33,2.7,1.7,2.84
2012/6/4,54677,7391.4,1943.3,230450,1607.64,16294.6,2.68,28.64,1878.8,1.25,509.1,6.32,2.69,1.7,2.2
2012/6/11,55026,7413.95,1920.25,242540,1614.69,16929.9,2.67,28.71,1892.35,1.26,500.82,6.37,2.68,1.7,2.2
2012/6/18,55664,7458.75,1859.3,251255,1597.63,16734.3,2.6,27.78,1852.55,1.26,500.02,6.36,2.65,1.7,2.2
2012/6/25,54952,7444.7,1829.95,254915,1576.45,16340.8,2.63,27.06,1803.1,1.25,506.14,6.36,2.66,1.7,2.2
2012/7/2,56115,7682.85,1903.05,254435,1603.17,16735.8,2.71,27.71,1871.3,1.25,497.8,6.36,2.65,1.4,1.8
2012/7/9,55613,7576.15,1872.85,252405,1578.11,16044.4,2.78,27.14,1849.05,1.22,492.5,6.37,2.63,1.4,1.8
2012/7/16,56125,7640.05,1888.55,252775,1581.88,16033.6,2.89,27.26,1863.85,1.23,487.34,6.37,2.62,1.4,1.8
2012/7/23,54968,7454.6,1847.5,251125,1600.06,15750.4,2.84,27.29,1808.75,1.22,488.84,6.39,2.63,1.4,1.8
2012/7/30,55100,7453.35,1834.9,247610,1604.91,15647.8,2.87,27.66,1825.55,1.23,482.56,6.37,2.62,1.58,1.92
2012/8/6,55369,7527.55,1859.55,244290,1614.14,15580.4,2.98,28.03,1844.95,1.24,476.48,6.36,2.62,1.7,2
2012/8/13,55228,7427.35,1810.55,236570,1607.92,15396,3.04,27.92,1782.4,1.23,482.48,6.36,2.61,1.7,2
2012/8/20,55881,7590.95,1848.99,235105,1649.13,16024,3.08,29.83,1814.8,1.25,482.22,6.35,2.61,1.7,2
2012/8/27,55945,7586.02,1878.32,232635,1666.09,16111.65,3.09,30.87,1832.1,1.25,480.18,6.35,2.61,1.7,2
2012/9/3,56481,7737.75,1954.5,217660,1703.22,16108.4,2.97,32.6,1880.7,1.26,479.47,6.34,2.61,2,1.9
2012/9/10,58765,8126.47,2101.18,215705,1744.54,16907,3.01,33.84,2011.55,1.29,473.2,6.33,2.61,2,1.9
2012/9/17,59965,8294.6,2132.2,216605,1768.32,17934.55,2.9,34.53,2080.25,1.3,469.8,6.31,2.6,2,1.9
2012/9/24,59160,8192.15,2068.6,220050,1765.18,18201.9,3.01,34.13,2066.9,1.29,471.88,6.3,2.6,2,1.9
2012/10/1,59152.5,8300.4,2084.55,223435,1779.67,18479,2.9,34.64,2052.6,1.29,472.56,6.31,2.6,2.2,1.7
2012/10/8,59121,8171.95,2003.75,219557.5,1764.47,17633.8,2.93,33.83,1945.81,1.29,473.84,6.29,2.59,2.2,1.7
2012/10/15,58743,8132.3,1952.5,218700,1738.87,17026.1,2.78,32.72,1868.55,1.3,471.81,6.25,2.58,2.2,1.7
2012/10/22,57459,7844.75,1915.2,221175,1711.26,16231.3,2.65,31.97,1805.65,1.3,479.36,6.25,2.58,2.2,1.7
2012/10/29,52508,7733.3,1889.2,240665,1705.74,16041.4,2.66,31.77,1822.65,1.29,480.85,6.24,2.6,2.04,1.82
2012/11/5,55964,7626.2,1894.25,245085,1715.95,15969,2.64,31.96,1863.3,1.28,479.84,6.25,2.61,1.8,2
2012/11/12,55616,7633.3,1948.05,252535,1721.57,15984.3,2.68,32.48,1912.45,1.27,482.78,6.23,2.6,1.8,2
2012/11/19,56040,7739.75,1941.5,252510,1733.18,16532,2.74,33.38,1912.55,1.29,478.82,6.23,2.6,1.8,2
2012/11/26,56197,7832.8,2027.65,249325,1730.02,16954.9,2.74,33.9,1989.2,1.3,480.26,6.23,2.58,1.8,2
2012/12/3,57137,8008.65,2098.65,251645,1702.05,17335.9,2.65,33.08,2006.8,1.3,479.03,6.23,2.58,1.7,2.5
2012/12/10,57357,8076.8,2129.15,266530,1705.29,17705.1,2.62,32.89,2059.38,1.31,474.61,6.25,2.57,1.7,2.5
2012/12/17,56990,7896.4,2070.4,305670,1668.04,17545.6,2.72,30.96,2050.85,1.32,474.65,6.23,2.56,1.7,2.5
2012/12/24,56580,7845.5,2040.45,317750,1659.01,17208,2.79,29.97,2049,1.32,478.9,6.23,2.55,1.7,2.5
2012/12/31,57222.5,8065.45,2072.42,320210,1671.13,17340.6,2.78,30.39,2060.8,1.31,475.22,6.23,2.55,1.62,2.1
2013/1/7,57657,8046,2048.82,326570,1660.16,17352.5,2.78,30.42,1992.85,1.32,470.92,6.22,2.55,1.6,2
2013/1/14,57563,7981.85,2011.3,340715,1679.47,17399.1,2.75,31.47,1980.35,1.33,473.3,6.22,2.54,1.6,2
2013/1/21,57922,8052.95,2026.75,344130,1678.62,17357.8,2.84,31.83,2051.45,1.34,470.53,6.22,2.55,1.6,2
2013/1/28,58317,8131.55,2048.75,366385,1665.06,18100.5,3.01,31.49,2109.6,1.36,471.35,6.22,2.57,1.68,2.24
2013/2/4,58928,8226.25,2069.23,386895,1672.42,18382.7,3.03,31.64,2155.15,1.35,472.3,6.23,2.58,2,3.2
2013/2/11,58822.5,8186.35,2096.75,399870,1637.2,18239.1,3.07,30.58,2166.2,1.34,470.94,6.23,2.57,2,3.2
2013/2/18,57930,7926.4,2049.15,414620,1587.07,17140.3,3.09,29.03,2104.85,1.33,472.49,6.24,2.58,2,3.2
2013/2/25,56986,7788.4,1968.1,443785,1591.64,16594.1,3.03,28.87,2056.9,1.31,473.09,6.23,2.59,1.9,2.98
2013/3/4,56433,7707.65,1926.2,479955,1577.9,16555.2,3.14,28.81,1967.9,1.3,472.58,6.22,2.6,1.5,2.1
2013/3/11,56757,7756.15,1926.78,520005,1588.55,16933.9,3.15,28.9,1943.05,1.3,471.13,6.21,2.59,1.5,2.1
2013/3/18,55769,7558.05,1899.4,552340,1608.75,16732.5,3.08,28.87,1900.2,1.29,472.16,6.21,2.59,1.5,2.1
2013/3/25,55765,7550.05,1879.78,567915,1599.26,16695.67,3.1,28.58,1875.73,1.28,471.9,6.21,2.59,1.5,2.1
2013/4/1,54696,7408.4,1851.22,574580,1573.67,16128.63,2.96,27.27,1835.92,1.29,470.84,6.2,2.58,1.1,2.4
2013/4/8,55284,7504.08,1858.9,587740,1551.08,16014.2,2.87,27.24,1868.07,1.31,467.64,6.2,2.58,1.1,2.4
2013/4/15,52404,7100.85,1865.2,611665,1378.58,15438.5,2.76,23.12,1847.8,1.31,474.05,6.18,2.58,1.1,2.4
2013/4/22,51010,6979.25,1874.2,618625,1439.18,15192.1,2.78,23.52,1867.25,1.3,474.76,6.17,2.61,1.1,2.4
2013/4/29,51246,6993.05,1820,615850,1468.21,15052.95,2.79,24.03,1829.95,1.31,471.26,6.16,2.64,1.28,2.22
2013/5/6,53484,7312.15,1855.33,605415,1459.68,15220.68,2.86,23.87,1842.23,1.31,470.96,6.15,2.61,1.4,2.1
2013/5/13,53468,7259.85,1823.95,621455,1398.45,14930.1,2.86,22.89,1809.4,1.29,478.15,6.15,2.62,1.4,2.1
2013/5/20,53853,7338.1,1825.7,624975,1382.99,14952.1,2.85,22.48,1827.25,1.29,486.13,6.13,2.66,1.4,2.1
2013/5/27,53488,7274.1,1842.18,614672.5,1393.61,14758.3,2.82,22.45,1857.2,1.29,491.99,6.13,2.69,1.4,2.1
2013/6/3,53802,7330.8,1918.75,612770,1401.39,15091.1,2.83,22.36,1909.35,1.31,501.42,6.13,2.73,1.8,2.7
2013/6/10,52862,7064.2,1835.27,610835,1385.24,14343.4,2.85,21.8,1825,1.33,498.26,6.13,2.74,1.8,2.7
2013/6/17,51362,6893.35,1775.96,641585,1333.9,14010.5,2.84,20.88,1812.85,1.33,503.41,6.13,2.76,1.8,2.7
2013/6/24,46208,6712.6,1728.17,671475,1243.27,13676.1,2.74,19.13,1808.85,1.3,507.13,6.14,2.78,1.8,2.7
2013/7/1,50298,6926.3,1765.65,661075,1242.95,13726,2.83,19.41,1838.85,1.3,504.16,6.13,2.78,2,2.7
2013/7/8,50119,6859.05,1773.9,647735,1263.32,13490.35,2.99,19.52,1856,1.3,507.17,6.13,2.77,2,2.7
2013/7/15,50608,6907.8,1766.2,639965,1285.75,13811,3.11,19.59,1835.2,1.31,500.36,6.14,2.76,2,2.7
2013/7/22,51165,6984.9,1787.5,627700,1333.76,14048.1,3.05,20.24,1841,1.32,503.63,6.14,2.78,2,2.7
2013/7/29,50355,6888.45,1753.2,612905,1317.95,13726.4,3.02,19.73,1810.95,1.33,513.12,6.13,2.79,1.8,2.66
2013/8/5,51312,7069.25,1773.15,601140,1299.51,14045.2,2.9,19.82,1844.65,1.33,511.37,6.12,2.79,1.5,2.6
2013/8/12,52734,7285.45,1855.02,587880,1345.24,14745.9,2.95,22.15,1922.95,1.33,509.18,6.12,2.79,1.5,2.6
2013/8/19,53212,7279.85,1852.57,569325,1374.46,14516.8,2.95,23.2,1941.85,1.34,514.28,6.12,2.81,1.5,2.6
2013/8/26,53180,7213.75,1809.61,577427.5,1408.46,14117.1,3.03,24.07,1915.97,1.33,511.88,6.12,2.81,1.5,2.6
2013/9/2,52606,7147.85,1765.16,600010,1390.35,13692.2,2.86,23.74,1852.4,1.32,508.69,6.12,2.8,1.2,3.1
2013/9/9,52392,7095.18,1754.6,585130,1352.63,13757.15,2.76,22.71,1837.1,1.33,504.47,6.12,2.78,1.2,3.1
2013/9/16,52087,7165.35,1748.55,570410,1334.62,13942.6,2.7,22.18,1836.05,1.34,502.62,6.12,2.75,1.2,3.1
2013/9/23,52450,7202.5,1769.5,548445,1327.14,13796.9,2.67,21.67,1855,1.35,500.05,6.12,2.76,1.2,3.1
2013/9/30,52716.67,7220.15,1790.02,531820,1311.24,13731.2,2.63,21.43,1847.6,1.36,501.84,6.12,2.77,1.04,3.02
2013/10/7,52504.33,7159.9,1821.93,515575,1300.86,13765.4,2.65,21.86,1853.15,1.35,498.97,6.12,2.78,1,3
2013/10/14,52434,7222.3,1807.9,502515,1294.28,14007.9,2.67,21.49,1891.8,1.36,496.34,6.1,2.76,1,3
2013/10/21,52179,7207,1814.5,488505,1337.03,14515.9,2.6,22.51,1909.25,1.38,503.01,6.09,2.76,1,3
2013/10/28,51886,7229.85,1829.65,476690,1335.21,14566.2,2.61,22.24,1925.05,1.37,508.6,6.09,2.76,1.04,2.96
2013/11/4,51723,7144.2,1774.4,469250,1307.81,14083.2,2.53,21.64,1878.4,1.35,515.73,6.09,2.79,1.2,2.8
2013/11/11,51278,7051.3,1751.1,457430,1281.73,13636.25,2.63,20.84,1857.75,1.35,520.22,6.09,2.8,1.2,2.8
2013/11/18,50655,7006.2,1739.45,444705,1255.78,13477.9,2.69,20.08,1864.4,1.35,520.47,6.09,2.8,1.2,2.8
2013/11/25,50964,7047.75,1716.65,431455,1244.94,13372.5,2.69,19.86,1857.15,1.36,526.23,6.09,2.8,1.2,2.8
2013/12/2,50804,7039.45,1713.35,414815,1227.88,13565.8,2.71,19.35,1870.45,1.36,530.95,6.09,2.8,1.5,2.5
2013/12/9,51263,7208.55,1756.72,399255,1243.44,13947.2,2.66,19.93,1953.15,1.38,530.17,6.07,2.78,1.5,2.5
2013/12/16,51259,7269.6,1744.27,385430,1216.12,14111.6,2.7,19.64,1996.35,1.37,528.35,6.07,2.76,1.5,2.5
2013/12/23,51781,7327.7,1734.3,372990,1205.84,14142.6,2.81,19.65,2084.75,1.37,527.25,6.07,2.78,1.5,2.5
2013/12/30,52219,7377.97,1754.63,364942.5,1213.41,13901.5,2.73,19.71,2075.3,1.37,526.95,6.05,2.79,1.56,2.5
2014/1/6,52069,7326.3,1726.15,350600,1234.18,13501.7,2.66,19.83,2029.1,1.36,531.87,6.05,2.8,1.6,2.5
2014/1/13,52032,7351.85,1745.53,337480,1246.96,14444.4,2.62,20.23,2074.5,1.36,531.31,6.05,2.8,1.6,2.5
2014/1/20,51592,7306.4,1738.58,329595,1252.7,14595.8,2.65,19.97,2057.05,1.36,545.15,6.05,2.81,1.6,2.5
2014/1/27,50813.83,7150.7,1693.3,319500,1253.48,13958.1,2.64,19.45,1979.75,1.36,549.59,6.06,2.82,1.6,2.5
2014/2/3,50911.67,7121.7,1656.76,311130,1258.77,13873.9,2.66,19.71,1984.2,1.35,556.12,6.06,2.82,1.1,2
2014/2/10,50682,7152.95,1683.52,302415,1295.5,14191.4,2.76,20.47,2017.75,1.36,550.55,6.06,2.81,1.1,2
2014/2/17,50672,7214.5,1718.45,290950,1321.67,14379.8,2.83,21.73,2058.35,1.37,551.39,6.08,2.8,1.1,2
2014/2/24,49468,7107.7,1721.93,277720,1332.55,14369.1,2.8,21.48,2077.8,1.37,557.93,6.12,2.8,1.1,2
2014/3/3,48908,7006.8,1721.1,272755,1342.16,15152.1,2.97,21.2,2117.85,1.38,561.71,6.13,2.8,1.5,2.4
2014/3/10,45170,6522.8,1710.4,256615,1360.42,15599,2.95,21.09,2005.35,1.39,572.41,6.14,2.8,1.5,2.4
2014/3/17,45252,6500.55,1683.42,265680,1342.22,16035.9,2.89,20.6,1956.05,1.38,567.18,6.2,2.81,1.5,2.4
2014/3/24,46099,6570,1698.77,267200,1301.66,15888.9,2.91,19.8,1959.9,1.38,555.74,6.21,2.81,1.5,2.4
2014/3/31,47081,6650.65,1774.2,261535,1287.58,16119.3,2.9,19.81,1975.3,1.38,553.06,6.21,2.81,1.9,1.92
2014/4/7,47490,6658.05,1820,251260,1310.32,16758.6,2.99,19.91,2013.7,1.38,547.52,6.21,2.78,2,1.8
2014/4/14,47877,6633.22,1832.78,242900,1303.94,17808.93,3.05,19.68,2046.25,1.38,555.07,6.22,2.77,2,1.8
2014/4/21,48513,6720.48,1838.42,240770,1290.56,18298.87,3.09,19.51,2060.85,1.38,560.4,6.24,2.79,2,1.8
2014/4/28,50021,6717.15,1759.95,233240,1293.28,18214.1,3,19.29,2043.2,1.39,562.65,6.26,2.8,2.04,2.08
2014/5/5,49373,6752.55,1728.72,218267.5,1296.74,18958.55,2.9,19.34,2035.53,1.39,561.34,6.23,2.79,2.1,2.5
2014/5/12,50362,6930.1,1741.2,196675,1296.51,19900.9,2.95,19.5,2066.3,1.37,551.06,6.23,2.78,2.1,2.5
2014/5/19,50189,6961.4,1742.7,179360,1292.88,19679.2,2.99,19.4,2074.2,1.37,551.46,6.24,2.79,2.1,2.5
2014/5/26,50231,6994.75,1797.62,173285,1263.36,19296.6,3.01,19.04,2072.65,1.36,551.51,6.24,2.77,2.1,2.5
2014/6/2,50027.5,6854.4,1826.85,168765,1247.44,19012.6,2.95,18.86,2091.7,1.36,550.48,6.25,2.78,2.1,2.3
2014/6/9,48828,6692.9,1837.2,166050,1264.36,18325.4,3.02,19.3,2098.65,1.35,553.44,6.23,2.79,2.1,2.3
2014/6/16,49260,6746.1,1829.6,160710,1290.57,18610,3.1,20.18,2126.85,1.36,556.59,6.23,2.8,2.1,2.3
2014/6/23,50533,6947.2,1858.6,156855,1317.26,18499.2,3.1,20.94,2177.95,1.36,550.65,6.23,2.8,2.1,2.3
2014/6/30,51712,7113.3,1881.6,155940,1323.61,19337.7,3.03,21.07,2219.9,1.37,550.29,6.21,2.79,2.02,2.3
2014/7/7,51584,7151.7,1907.35,158610,1327.76,19376.2,2.95,21.18,2279.7,1.36,552.33,6.2,2.78,2,2.3
2014/7/14,50850,7074.45,1951.1,158845,1305.63,19091.2,2.89,20.84,2293.4,1.35,560.07,6.21,2.78,2,2.3
2014/7/21,50304,7084.7,2002.55,155675,1304.86,18998.8,2.87,20.75,2367.25,1.35,563.69,6.2,2.78,2,2.3
2014/7/28,50604,7123.3,1983.45,148250,1294.51,18597.9,2.82,20.47,2380.05,1.34,568.78,6.18,2.79,1.94,2.24
2014/8/4,50484,7041.1,2012.6,144415,1300.71,18544.5,2.74,19.94,2347.85,1.34,575.14,6.17,2.8,1.7,2
2014/8/11,49931,6924.55,2008.47,141210,1309.13,18576.9,2.72,19.81,2283.1,1.34,575.97,6.15,2.79,1.7,2
2014/8/18,50176,7002.6,2042.8,144915,1288.03,18630.9,2.71,19.45,2326.75,1.33,581.11,6.14,2.81,1.7,2
2014/8/25,50875,7038.65,2078.58,147290,1282.96,18734.9,2.76,19.41,2351.6,1.32,587.61,6.14,2.83,1.7,2
2014/9/1,50462,6963.45,2076.2,152015,1270.05,18988.7,2.6,19.2,2374.15,1.3,589.36,6.14,2.85,1.7,1.6
2014/9/8,49938,6895.45,2022.65,155435,1245.48,18795.5,2.54,18.86,2297.55,1.29,589.94,6.13,2.85,1.7,1.6
2014/9/15,49515,6911.35,1958.15,155450,1226.13,17898.1,2.57,18.43,2258.8,1.29,595.3,6.14,2.86,1.7,1.6
2014/9/22,48706,6766,1929.15,154400,1218.81,17100.7,2.65,17.65,2252.85,1.28,598.88,6.14,2.87,1.7,1.6
2014/9/29,48759,6713.6,1898.25,152170,1208.48,16260.3,2.48,17.09,2265.75,1.26,597.74,6.14,2.89,1.7,1.6
2014/10/6,48625.5,6725.65,1910.77,149855,1216.82,16573.9,2.33,17.31,2312.1,1.27,594.43,6.14,2.9,1.7,1.6
2014/10/13,48424,6724.2,1914.45,154620,1237.21,15883.2,2.21,17.37,2281.15,1.28,588.07,6.12,2.9,1.7,1.6
2014/10/20,47875,6705.35,1978.85,157985,1239.95,15139.6,2.19,17.28,2238.7,1.27,584.33,6.12,2.9,1.7,1.6
2014/10/27,48236,6825.1,2021.9,161700,1207.36,15450,2.19,16.8,2288.55,1.26,578.56,6.11,2.91,1.7,1.6
2014/11/3,48135,6746.25,2068.55,160945,1158.68,15393.9,2.11,15.73,2248.2,1.25,587.1,6.12,2.92,1.3,1.4
2014/11/10,48068,6746.5,2047.87,160815,1165.61,15393.4,2.07,15.78,2253.9,1.25,592.08,6.12,2.93,1.3,1.4
2014/11/17,48041,6750.55,2039.25,160540,1192.03,16063.6,2.04,16.22,2253.2,1.25,597.07,6.12,2.93,1.3,1.4
2014/11/24,47788,6608.1,2066.55,162685,1190.13,16377.3,1.99,16.26,2257.6,1.25,600.36,6.14,2.91,1.3,1.4
2014/12/1,46458,6494.6,1993.35,164760,1203.16,16565.7,1.81,16.4,2222.05,1.24,611.34,6.15,2.94,0.8,1.5
2014/12/8,46874,6505.8,1936.83,165955,1221.93,16430.1,1.66,16.9,2187.85,1.24,614.83,6.18,2.96,0.8,1.5
2014/12/15,46535,6418.1,1885.55,170640,1194.63,15794.2,1.56,15.89,2139.25,1.23,616.15,6.2,2.95,0.8,1.5
2014/12/22,46483,6375.6,1839.5,171295,1179.29,15288,1.53,15.78,2150.05,1.22,608.1,6.22,2.97,0.8,1.5
2014/12/29,46295,6360.5,1821.97,175035,1187.85,14942,1.44,15.83,2170.68,1.21,608.2,6.2,2.98,0.44,1.22
2015/1/5,45679,6191.85,1781.1,180515,1213.17,15304.3,1.35,16.39,2150.45,1.18,614.65,6.21,2.98,-0.1,0.8
2015/1/12,42767,5821.7,1791.52,194370,1246.98,14598.1,1.31,17.03,2073.6,1.17,622.66,6.2,2.99,-0.1,0.8
2015/1/19,41756,5695.5,1842.95,221360,1292.09,14638.9,1.33,18.07,2104.9,1.15,626.66,6.22,3.01,-0.1,0.8
2015/1/26,40240,5515,1845.2,243065,1279.43,14874.1,1.36,17.61,2106.9,1.13,625.64,6.25,3.04,-0.1,0.8
2015/2/2,41199,5678.05,1863.8,264285,1260.27,15180.25,1.54,17.15,2138.3,1.14,626.19,6.25,3.06,0,1.4
2015/2/9,41263,5681.75,1825.23,290435,1228.38,14758.2,1.58,16.97,2126.75,1.13,624.03,6.24,3.07,0,1.4
2015/2/16,41863,5731.5,1795.15,297100,1212.02,14148.2,1.61,16.57,2078.75,1.14,619.2,6.25,3.08,0,1.4
2015/2/23,42162.5,5825.8,1787.1,296325,1205.75,14208.7,1.69,16.44,2047.25,1.13,618.78,6.26,3.09,0,1.4
2015/3/2,42585,5850.25,1782.4,308830,1195.05,13947.4,1.91,16.19,2011.2,1.11,619.02,6.27,3.09,-0.1,1.4
2015/3/9,42293,5837.2,1750.45,329850,1158.77,14016.7,1.82,15.62,1998.95,1.06,634.4,6.26,3.09,-0.1,1.4
2015/3/16,42275,5863.5,1778.05,340955,1164.73,13791.8,1.77,15.98,2006.85,1.07,637.49,6.23,3.08,-0.1,1.4
2015/3/23,43572,6149.4,1779.45,340520,1195.9,13735.6,1.82,16.96,2075.2,1.09,622.1,6.21,3.07,-0.1,1.4
2015/3/30,43575,6054.52,1782.28,333216.67,1195.21,12723.23,1.79,16.75,2103.08,1.08,619.31,6.2,3.09,-0.16,1.46
2015/4/6,43593,6048.13,1773.57,331613.33,1205.43,12548.27,1.8,16.59,2161.57,1.08,613.36,6.2,3.1,-0.2,1.5
2015/4/13,43589,6010.7,1813,336475,1198.89,12568.8,1.89,16.25,2200.55,1.07,612.74,6.2,3.11,-0.2,1.5
2015/4/20,43714,5960.95,1825.05,338250,1191.63,12764.3,1.95,15.88,2213.65,1.08,615.24,6.2,3.11,-0.2,1.5
2015/4/27,44420,6228.1,1890.55,338470,1196.05,13588.1,2.02,16.35,2332.3,1.11,609.57,6.2,3.12,-0.16,1.44
2015/5/4,45950,6422.42,1901,340320,1188.77,14106.15,2.02,16.44,2392.1,1.12,609.53,6.21,3.14,0,1.2
2015/5/11,45979,6409.2,1841.3,338530,1207.22,14050.7,2.04,16.96,2325.25,1.13,601.27,6.2,3.14,0,1.2
2015/5/18,45579,6239.6,1748.25,334540,1210.74,13089.9,2.04,17.22,2208.35,1.11,603.7,6.2,3.14,0,1.2
2015/5/25,44620,6081.9,1714.8,326367.5,1191.89,12680.25,2.01,16.79,2185.75,1.1,614.27,6.2,3.14,0,1.2
2015/6/1,43740,5975.45,1711.25,318060,1182.89,13002.5,2.03,16.44,2141.7,1.11,627.43,6.2,3.14,0.1,1.4
2015/6/8,43480,5935.05,1711.8,312530,1179.48,13357.8,2.1,15.98,2124.25,1.13,629.13,6.21,3.15,0.1,1.4
2015/6/15,42456,5722.65,1664.8,317545,1190.71,12733.4,2.1,16.09,2062.9,1.13,632.64,6.21,3.15,0.1,1.4
2015/6/22,42386,5724.35,1675.85,312155,1177.17,12593.3,2.05,15.91,2029.9,1.12,632.29,6.21,3.17,0.1,1.4
2015/6/29,42739,5765.3,1671.7,319850,1171.07,11963.2,2.03,15.67,2016.1,1.11,637.66,6.2,3.17,0.16,1.52
2015/7/6,41353,5522.1,1644.45,327740,1160.68,11169.25,1.99,15.37,1985.5,1.11,647.16,6.21,3.17,0.2,1.6
2015/7/13,41129,5524.9,1668,336940,1148.11,11556.4,1.91,15.16,2055.5,1.09,643.04,6.21,3.17,0.2,1.6
2015/7/20,40055,5346.4,1614.05,340430,1096.27,11468.7,1.88,14.72,1999.05,1.09,654.22,6.21,3.18,0.2,1.6
2015/7/27,39308,5250.1,1605.8,344740,1093.59,11086.7,1.82,14.7,1949.45,1.1,668.53,6.21,3.18,0.2,1.6
2015/8/3,38590,5190.35,1566.8,349755,1088.08,10770.7,1.66,14.62,1880.9,1.09,679.94,6.21,3.19,0.2,2
2015/8/10,39225,5183.05,1558,351895,1113.22,10674.5,1.71,15.34,1829.75,1.11,681.54,6.34,3.21,0.2,2
2015/8/17,39051,5065.1,1533.5,353180,1136.25,10366.8,1.59,15.28,1784.05,1.12,692.8,6.39,3.24,0.2,2
2015/8/24,39134,5070.7,1534.2,364490,1135.63,9732,1.45,14.52,1735.85,1.14,699.52,6.4,3.29,0.2,2
2015/8/31,39329,5145.98,1588.95,363290,1130.69,9869.6,1.46,14.63,1800.78,1.12,691.04,6.36,3.22,0.04,1.68
2015/9/7,40399,5343.35,1607.05,346335,1112.97,10093,1.39,14.63,1798.85,1.12,689.42,6.37,3.21,0,1.6
2015/9/14,40934,5351.25,1609.5,336300,1120.2,9929.3,1.35,14.79,1712.95,1.13,680.28,6.37,3.19,0,1.6
2015/9/21,39478,5109.55,1574.9,328260,1137.3,9799.2,1.39,14.98,1632.75,1.12,696.75,6.38,3.2,0,1.6
2015/9/28,38496.5,5073,1551.05,322440,1124.72,10012,1.36,14.68,1651.8,1.12,697.94,6.37,3.22,0.08,1.48
2015/10/5,39264,5207.55,1558.7,309725,1144.16,10111.4,1.41,15.8,1681.6,1.13,679.36,6.35,3.21,0.2,1.3
2015/10/12,39996,5312.3,1564.8,297515,1174.95,10515.85,1.32,15.99,1797.9,1.14,675.33,6.34,3.22,0.2,1.3
2015/10/19,39339,5204.05,1484.6,284915,1168.34,10370.7,1.28,15.8,1733.2,1.12,687.05,6.35,3.24,0.2,1.3
2015/10/26,39036,5182.65,1444.35,272670,1154.38,10394.4,1.34,15.73,1707.45,1.1,688.61,6.35,3.27,0.2,1.3
2015/11/2,38572,5088.3,1483.15,266565,1110.43,9837.3,1.39,15.1,1651.7,1.09,692.78,6.34,3.29,0.5,1.5
2015/11/9,37450,4906.6,1492.65,259150,1087.32,9482.4,1.31,14.36,1602.55,1.08,701.2,6.37,3.31,0.5,1.5
2015/11/16,35234,4656.65,1450.65,256680,1076.63,8976.7,1.27,14.19,1530.6,1.07,712.1,6.38,3.34,0.5,1.5
2015/11/23,34435,4588.2,1451.25,247575,1069.08,8756.9,1.38,14.16,1556,1.06,712.83,6.39,3.36,0.5,1.5
2015/11/30,35126,4596.4,1469.85,240695,1066.94,8901.8,1.32,14.17,1533.4,1.07,703.87,6.4,3.36,0.66,1.58
2015/12/7,35274,4604.5,1482.95,233755,1072.8,8632.7,1.24,14.11,1517.3,1.09,706.59,6.43,3.37,0.7,1.6
2015/12/14,35832,4609.9,1490.25,231985,1061.94,8646.4,1.25,13.87,1498.1,1.09,706.49,6.47,3.36,0.7,1.6
2015/12/21,36200,4699.15,1529.2,233845,1074.5,8662.27,1.23,14.3,1527.97,1.1,697.44,6.48,3.38,0.7,1.6
2015/12/28,36363,4708.25,1512.28,236547.5,1064.33,8688.23,1.26,13.88,1587.33,1.09,708.11,6.49,3.4,0.84,1.64
2016/1/4,36178,4576,1470.1,236305,1091.57,8503.8,1.2,14,1524.6,1.08,718.53,6.56,3.42,1.4,1.8
2016/1/11,35046,4371,1467.35,235690,1088.01,8338.5,1.07,13.9,1475.45,1.09,729.21,6.58,3.42,1.4,1.8
2016/1/18,34969,4406,1480.75,236090,1095.03,8604.65,1.04,14.02,1496.25,1.09,725.3,6.58,3.44,1.4,1.8
2016/1/25,35236,4534.15,1506.2,240030,1117,8582.2,1.06,14.32,1585.15,1.09,715.92,6.58,3.46,1.4,1.8
2016/2/1,35518,4617.8,1517.1,235035,1145.5,8378,1.02,14.61,1685.05,1.11,706.24,6.58,3.48,1,2.3
2016/2/8,35742.5,4507.65,1489.7,225500,1211.68,7885.9,0.96,15.45,1708.55,1.13,710.78,6.57,3.5,1,2.3
2016/2/15,35446,4584.4,1530.5,212190,1214.56,8367.7,0.98,15.3,1686.55,1.11,702.65,6.51,3.5,1,2.3
2016/2/22,35813,4663.25,1574.75,201625,1223.52,8523.9,1.01,15.09,1749.45,1.1,692.32,6.53,3.52,1,2.3
2016/2/29,36251,4826.45,1598.4,190855,1246.48,8826.25,1.26,15.06,1807.8,1.09,687.58,6.54,3.5,0.92,2.3
2016/3/7,37161,4943.8,1571,181320,1260.64,8859.1,1.43,15.45,1781.35,1.11,680.48,6.51,3.44,0.9,2.3
2016/3/14,37478,5011.5,1505.35,166095,1248.1,8626.9,1.42,15.57,1784.85,1.12,679.51,6.49,3.35,0.9,2.3
2016/3/21,37766,5020.3,1473.65,151985,1228.86,8672.3,1.47,15.45,1828.05,1.12,677.63,6.5,3.38,0.9,2.3
2016/3/28,37260,4901.9,1493.65,146520,1228.2,8406.17,1.44,15.24,1803.05,1.13,674.2,6.48,3.35,0.94,2.3
2016/4/4,36565,4741.15,1499.5,143330,1229.58,8435.2,1.4,15.12,1786.1,1.14,677.78,6.48,3.36,1.1,2.3
2016/4/11,36540,4798.35,1531.4,146950,1242.91,8829.75,1.51,16.12,1831.52,1.13,670.14,6.47,3.28,1.1,2.3
2016/4/18,37214,4971.35,1611.2,146450,1241.08,9153.1,1.5,16.8,1906.85,1.13,663.74,6.47,3.25,1.1,2.3
2016/4/25,37524,4982.7,1652.4,151460,1256.99,9217.55,1.57,17.33,1895.25,1.13,665.26,6.48,3.28,1.1,2.3
2016/5/2,37052,4886.32,1609.65,155802.5,1284.45,9265.73,1.51,17.4,1884.05,1.15,665.9,6.49,3.31,1,2
2016/5/9,35812,4674.15,1534.3,159760,1268.15,8652,1.54,17.1,1862.2,1.14,681.97,6.51,3.32,1,2
2016/5/16,35558,4622.8,1534.2,157390,1263.37,8599.25,1.63,16.83,1874.35,1.12,693.24,6.53,3.32,1,2
2016/5/23,35574,4654.75,1544.2,155445,1226.38,8335.5,1.64,16.27,1853.2,1.12,690.84,6.56,3.34,1,2
2016/5/30,35658,4670.17,1541.58,153202.5,1217.27,8417.7,1.62,16.05,1954.33,1.12,689.21,6.57,3.35,1,1.94
2016/6/6,35874,4563.05,1565.9,200365,1258.62,8770.6,1.6,16.88,2044.65,1.13,679.07,6.56,3.31,1,1.9
2016/6/13,35618,4546.7,1604.5,201600,1287.15,8902.85,1.5,17.38,2010.5,1.13,686.09,6.59,3.32,1,1.9
2016/6/20,36064,4690.2,1623.38,192730,1278.93,9143.65,1.58,17.39,2022.3,1.13,675.34,6.59,3.28,1,1.9
2016/6/27,37134,4815.5,1624.85,191900,1323.3,9389.7,1.5,18.44,2080.7,1.11,666.3,6.64,3.29,0.96,1.88
2016/7/4,37448,4755.95,1641.8,210765,1359.18,9858.85,1.41,20.04,2109.95,1.11,661.43,6.68,3.27,0.8,1.8
2016/7/11,37792,4863.95,1657.35,231320,1340.3,10263.75,1.4,20.23,2176.75,1.11,656.53,6.69,3.28,0.8,1.8
2016/7/18,38274,4944.2,1613.5,226970,1325.72,10536.6,1.37,19.74,2235.75,1.1,650.44,6.69,3.3,0.8,1.8
2016/7/25,37916,4888.6,1598.3,213535,1332.1,10459.5,1.32,19.97,2216.9,1.11,660.51,6.66,3.35,0.8,1.8
2016/8/1,37840,4840.6,1623.1,208605,1354.06,10662.25,1.34,20.28,2265,1.11,656.14,6.64,3.33,1.1,1.3
2016/8/8,37466,4794.8,1635.85,204300,1339.02,10645.45,1.35,19.86,2266.45,1.11,649.56,6.64,3.31,1.1,1.3
2016/8/15,37208,4779.5,1671.45,209975,1345.51,10290.55,1.45,19.64,2271.85,1.13,655.23,6.64,3.3,1.1,1.3
2016/8/22,36752,4652.3,1641.1,251920,1328.3,9983.55,1.5,18.67,2298.2,1.13,668.79,6.65,3.35,1.1,1.3
2016/8/29,36500,4607.3,1599.73,295485,1316.42,9821.4,1.38,18.86,2332.42,1.12,675.23,6.68,3.38,1.26,1.54
2016/9/5,36748,4621.4,1569.1,338385,1337.16,10171.9,1.35,19.58,2315.6,1.12,666.8,6.67,3.36,1.5,1.9
2016/9/12,36812,4699.35,1557,351460,1318.31,9791.75,1.4,18.91,2229.9,1.12,672.76,6.67,3.39,1.5,1.9
2016/9/19,37488,4786.05,1592.4,349660,1327.27,10379,1.39,19.53,2267.2,1.12,665.42,6.67,3.36,1.5,1.9
2016/9/26,37834,4812.45,1652.7,371215,1324.27,10527.95,1.45,19.19,2330.9,1.12,659.86,6.67,3.37,1.5,1.9
2016/10/3,37820,4772,1667.95,364075,1271.64,10143.15,1.49,17.81,2348.05,1.12,664.27,6.67,3.39,1.6,2.1
2016/10/10,37964,4751.75,1680.65,348620,1254.73,10441.45,1.48,17.47,2254.05,1.11,668.29,6.72,3.4,1.6,2.1
2016/10/17,37454,4642,1627.05,347955,1262.96,10177.55,1.51,17.54,2269,1.1,666.9,6.75,3.38,1.6,2.1
2016/10/24,38120,4737.35,1674.35,334565,1269.51,10241.35,1.49,17.65,2346.5,1.09,653.75,6.78,3.36,1.6,2.1
2016/10/31,38918,4915.9,1727,314980,1293.29,10388.2,1.44,18.28,2445.85,1.11,651.75,6.76,3.37,1.68,2.26
2016/11/7,42342,5365.15,1742.65,283685,1264.01,11299.5,1.35,18.18,2472.25,1.09,654.97,6.79,3.38,1.7,2.3
2016/11/14,44880,5480.85,1719.7,259075,1219.82,11149.85,1.32,16.83,2547.4,1.07,674.03,6.87,3.41,1.7,2.3
2016/11/21,46362,5716.6,1761.75,240365,1196.33,11455.85,1.4,16.47,2664.45,1.06,676.57,6.9,3.41,1.7,2.3
2016/11/28,47092,5783,1726.15,235050,1180.58,11273.45,1.48,16.56,2724.7,1.06,673.49,6.89,3.4,1.86,2.22
2016/12/5,47162,5840.65,1728.85,224580,1168.57,11399.4,1.52,16.86,2723.2,1.07,656.6,6.88,3.41,2.1,2.1
2016/12/12,46768,5699.5,1740.7,276045,1144.97,11264.95,1.54,16.55,2740.35,1.05,659.2,6.92,3.39,2.1,2.1
2016/12/19,45008,5487.1,1731.7,339890,1132.58,10698.7,1.6,15.87,2596.5,1.04,675.19,6.95,3.39,2.1,2.1
2016/12/26,44638,5502.45,1712.45,329460,1144.24,10092.25,1.66,15.93,2552.45,1.05,670.91,6.95,3.36,2.1,2.1
2017/1/2,45320,5550,1705.15,306325,1165.76,10072.1,1.64,16.32,2567.6,1.05,668.64,6.93,3.37,2.5,2.5
2017/1/9,46388,5738.65,1771.45,288785,1190.36,10330.1,1.59,16.71,2702.35,1.06,666.17,6.92,3.37,2.5,2.5
2017/1/16,46780,5751.75,1825.1,276385,1207.43,9993.7,1.57,17.01,2740.15,1.07,658.94,6.87,3.33,2.5,2.5
2017/1/23,47208.33,5868.65,1839.35,270115,1201.11,9561.7,1.55,17.03,2773.55,1.07,650.77,6.87,3.29,2.5,2.5
2017/1/30,46986.67,5875.1,1812.45,260195,1210.25,10045.45,1.54,17.42,2825.95,1.08,644.71,6.87,3.26,2.62,1.48
2017/2/6,46880,5876.9,1832.95,252325,1234.32,10397.3,1.54,17.74,2840.55,1.07,643.82,6.87,3.27,2.7,0.8
2017/2/13,48976,6016.75,1878.9,238560,1231.95,10854.8,1.54,17.94,2870.25,1.06,640.92,6.86,3.25,2.7,0.8
2017/2/20,48254,5977.85,1877.2,217940,1243.3,10796.15,1.51,18.1,2846.45,1.06,642.45,6.87,3.24,2.7,0.8
2017/2/27,47840,5943.45,1909.65,201425,1243.76,10911.05,1.6,18.13,2802.2,1.06,651.71,6.88,3.27,2.52,0.86
2017/3/6,47010,5746.9,1864.45,287710,1210.84,10340.5,1.64,17.28,2694.75,1.06,662.21,6.9,3.29,2.4,0.9
2017/3/13,47188,5844.9,1874.4,336080,1214.88,10158.4,1.59,17.16,2783.75,1.07,664.88,6.9,3.26,2.4,0.9
2017/3/20,46900,5791.4,1917.35,323285,1242.68,9986.4,1.6,17.54,2819,1.08,660.17,6.88,3.24,2.4,0.9
2017/3/27,46826,5844.75,1944.75,302720,1249.99,9929,1.66,18.15,2797.55,1.08,662.72,6.88,3.24,2.4,0.9
2017/4/3,47356,5794.4,1940.1,274680,1253.84,10014.4,1.72,18.19,2700.85,1.07,657.34,6.89,3.24,2.2,1.2
2017/4/10,46430,5666.98,1896.85,259776.67,1277.72,9764.68,1.75,18.33,2582.45,1.06,651.62,6.89,3.25,2.2,1.2
2017/4/17,46024,5566.42,1900.2,263153.33,1283.74,9334.77,1.68,18.14,2552.1,1.07,648.68,6.88,3.24,2.2,1.2
2017/4/24,46094,5674.05,1933.35,261775,1268.04,9269.7,1.59,17.48,2601.2,1.09,662.1,6.89,3.24,2.2,1.2
2017/5/1,46159,5631.75,1910.38,293560,1241.07,9230.1,1.51,16.52,2599.75,1.09,669.43,6.9,3.25,1.9,1.5
2017/5/8,44956,5496.65,1870.5,339935,1223.49,9185.3,1.54,16.25,2583.95,1.09,674.65,6.9,3.28,1.9,1.5
2017/5/15,45146,5601.55,1921.5,332850,1245.51,9173.8,1.61,16.74,2552.35,1.11,669.2,6.88,3.26,1.9,1.5
2017/5/22,45808,5678.1,1945.1,325615,1258.21,9150.2,1.64,17.17,2629.65,1.12,671.38,6.88,3.27,1.9,1.5
2017/5/29,45478,5648.32,1927.5,311365,1268.32,8943.45,1.61,17.38,2578.75,1.12,672.54,6.83,3.27,1.78,1.5
2017/6/5,45238,5651.75,1900.21,291930,1280.83,8823.95,1.52,17.46,2460.85,1.13,666.92,6.8,3.26,1.6,1.5
2017/6/12,45760,5674.45,1869.55,273000,1259.87,8805.95,1.46,16.79,2481.55,1.12,662.79,6.8,3.27,1.6,1.5
2017/6/19,45624,5710.95,1865.45,260390,1247.71,8934.4,1.43,16.5,2624.5,1.12,662.73,6.83,3.26,1.6,1.5
2017/6/26,46712,5866.8,1889.4,249380,1245.21,9202,1.48,16.63,2741.7,1.13,663.27,6.8,3.24,1.6,1.5
2017/7/3,46960,5843.2,1922.5,299930,1221.39,9105.05,1.52,15.96,2789.85,1.14,664.69,6.8,3.25,1.7,1.4
2017/7/10,46900,5856.3,1890.55,317960,1219.23,9180.8,1.53,15.79,2803.8,1.14,662.44,6.79,3.25,1.7,1.4
2017/7/17,47558,5954.05,1895.9,308065,1243.16,9560.6,1.59,16.27,2758.6,1.16,654.21,6.76,3.24,1.7,1.4
2017/7/24,49028,6216.7,1904.35,303335,1258.46,9983.7,1.62,16.57,2796.95,1.17,649.54,6.75,3.24,1.7,1.4
2017/7/31,50188,6329.05,1893.15,298250,1266,10251.7,1.66,16.59,2785.1,1.18,650.48,6.72,3.23,1.86,1.72
2017/8/7,50360,6407.85,2010.35,290260,1274.03,10644.6,1.62,16.75,2898.65,1.18,648.37,6.68,3.24,1.9,1.8
2017/8/14,50354,6423.3,2067.25,278090,1281.32,10602.2,1.59,16.93,3024.9,1.18,646.69,6.68,3.24,1.9,1.8
2017/8/21,51438,6589.5,2089.95,250560,1288.57,11469.7,1.62,17,3097.2,1.18,638.8,6.66,3.23,1.9,1.8
2017/8/28,52592,6757.25,2088.8,228990,1314.49,11683.25,1.85,17.49,3119.17,1.19,626.81,6.59,3.23,1.96,1.76
2017/9/4,53060,6835.55,2080.8,215420,1340.33,11985.15,1.68,17.91,3117.75,1.19,619.52,6.51,3.23,2.2,1.6
2017/9/11,51014,6556.9,2079.7,250820,1325.96,11400.1,1.65,17.74,3042.15,1.19,623.45,6.54,3.23,2.2,1.6
2017/9/18,50476,6460.15,2117.7,307540,1301.28,10963.2,1.66,17.1,3119.25,1.2,624.08,6.58,3.24,2.2,1.6
2017/9/25,50758,6410.6,2103.85,301605,1290.6,10363.3,1.66,16.83,3174.65,1.18,635.25,6.64,3.26,2.2,1.6
2017/10/2,51990,6532.2,2124.05,297120,1272.16,10464.4,1.58,16.62,3320.4,1.17,632.62,6.65,3.26,2,1.9
2017/10/9,53014,6759.55,2124.8,287500,1292.02,11183.9,1.59,17.15,3316.6,1.18,627.02,6.59,3.25,2,1.9
2017/10/16,54944,6984.5,2119.2,289490,1285.93,11688,1.64,17.07,3167.75,1.18,623.5,6.61,3.23,2,1.9
2017/10/23,55334,6946.05,2153.55,278520,1274.88,11763.7,1.73,16.89,3240.9,1.17,632.67,6.64,3.23,2,1.9
2017/10/30,54186,6863.4,2154.67,273235,1273.21,12379,1.77,16.91,3293.1,1.16,634.54,6.62,3.24,2.12,1.78
2017/11/6,53982,6817.4,2102.85,263880,1279.76,12484.4,1.82,17.01,3239.65,1.16,631.73,6.63,3.24,2.2,1.7
2017/11/13,53226,6752.65,2081.52,254355,1281.88,11719.4,1.75,17.08,3203.7,1.18,629.63,6.63,3.24,2.2,1.7
2017/11/20,53540,6903,2087.3,226945,1285.47,11749.5,1.77,17.02,3239.3,1.18,634.39,6.61,3.23,2.2,1.7
2017/11/27,53264,6793.2,2070.8,192930,1285.16,11308.5,1.75,16.65,3202.7,1.19,643.28,6.61,3.23,2.18,1.72
2017/12/4,52184,6579.6,2014.7,191555,1260.08,10949.5,1.7,15.98,3121.65,1.18,652.72,6.62,3.23,2.1,1.8
2017/12/11,52126,6713.75,2013.5,193285,1249.6,11169.2,1.68,15.88,3165.5,1.18,645.82,6.61,3.25,2.1,1.8
2017/12/18,53596,6981.15,2111.45,200570,1265.88,11907,1.72,16.17,3218.05,1.19,620.67,6.59,3.26,2.1,1.8
2017/12/25,54754,7190.05,2239.05,202075,1288.38,12234,1.79,16.66,3294.15,1.19,616.81,6.54,3.23,2.1,1.8
2018/1/1,54753,7133.98,2228.35,201212.5,1314.78,12529.2,1.79,17.11,3367.7,1.2,606.61,6.5,3.21,2.1,1.5
2018/1/8,54506,7085.85,2165.6,203410,1321.75,12653.8,1.83,17.03,3392.25,1.2,605.93,6.5,3.21,2.1,1.5
2018/1/15,53906,7048.5,2210.25,205860,1332.44,12547.95,1.86,17.08,3433.55,1.22,605.68,6.42,3.21,2.1,1.5
2018/1/22,53272,7032.35,2241.25,269050,1346.15,13270.8,1.91,17.24,3471.75,1.24,603.47,6.36,3.21,2.1,1.5
2018/1/29,53004,7042.1,2215,306035,1340.74,13603.5,1.9,17.06,3579.7,1.24,602.42,6.31,3.21,2.14,2.06
2018/2/5,52236,6902.55,2168.15,309825,1323.23,13244.2,1.78,16.47,3482.15,1.23,601.45,6.29,3.25,2.2,2.9
2018/2/12,51961,7037.5,2161.65,333360,1340.49,13700.3,1.71,16.67,3537,1.24,595.07,6.34,3.26,2.2,2.9
2018/2/19,52440,7078.2,2212.35,336085,1331.96,13681.8,1.77,16.53,3567.75,1.23,593.12,6.34,3.24,2.2,2.9
2018/2/26,52462,6941.6,2156.35,329140,1321.75,13647.1,1.84,16.46,3467.95,1.23,593.62,6.33,3.25,2.28,2.58
2018/3/5,51700,6897.4,2104.05,316840,1324.79,13513.2,1.91,16.51,3272.6,1.23,601.81,6.33,3.25,2.4,2.1
2018/3/12,51562,6899.15,2069.3,320640,1320.78,13675,1.91,16.45,3249.5,1.23,604.28,6.32,3.26,2.4,2.1
2018/3/19,50624,6714.95,2051.45,320180,1327,13260.2,1.99,16.38,3226.05,1.23,607.99,6.32,3.24,2.4,2.1
2018/3/26,49362,6636.43,2003.38,378206.67,1334.66,13098.73,2.01,16.42,3285.13,1.23,604.39,6.28,3.22,2.4,2.1
2018/4/2,50484,6739.57,1981.32,377548.33,1333.24,13274.77,1.97,16.39,3263.07,1.23,603.73,6.3,3.22,2.5,1.8
2018/4/9,50818,6838.9,2257.85,365630,1341.73,13686,2.04,16.54,3172.5,1.23,598.87,6.28,3.23,2.5,1.8
2018/4/16,51174,6915.4,2467.4,355565,1344.56,14700.2,2.07,16.97,3189,1.24,595.58,6.28,3.21,2.5,1.8
2018/4/23,51752,6905.1,2254.7,345360,1323.56,14045.5,2.11,16.56,3160.2,1.22,602.7,6.32,3.23,2.5,1.8
2018/4/30,50988,6768.7,2296.4,322435,1310.17,13764.85,2.1,16.33,3046.9,1.2,616.84,6.35,3.25,2.74,1.8
2018/5/7,50994,6804.68,2348.12,293760,1316.17,13880.7,2.16,16.53,3058.2,1.19,627.93,6.35,3.27,2.8,1.8
2018/5/14,50996,6820.6,2310.4,294765,1295.57,14481.1,2.22,16.38,3066.95,1.18,631.55,6.37,3.27,2.8,1.8
2018/5/21,51340,6871.8,2266.35,298930,1296.64,14697.7,2.24,16.5,3051.5,1.17,626.9,6.38,3.27,2.8,1.8
2018/5/28,51242,6851.78,2277.73,300340,1297.67,15046.35,2.16,16.41,3096.42,1.16,628.52,6.41,3.27,2.82,1.82
2018/6/4,52704,7186.85,2313.85,309735,1295.9,15498.35,2.11,16.59,3201.2,1.18,630.63,6.4,3.26,2.9,1.9
2018/6/11,53422,7181.6,2271.15,295385,1295.29,15244.9,2.08,16.89,3211.6,1.17,635.07,6.41,3.27,2.9,1.9
2018/6/18,52008,6829.4,2183.85,302290,1271.36,14887.2,2.04,16.34,3025,1.16,639,6.48,3.27,2.9,1.9
2018/6/25,51346,6682.2,2167.15,300035,1255.35,14737.2,2.11,16.13,2913.85,1.16,645.68,6.59,3.27,2.9,1.9
2018/7/2,50340,6404.45,2108.8,282620,1252.62,14165.2,2.12,15.98,2795.2,1.17,654.15,6.64,3.28,2.9,2.1
2018/7/9,48930,6252.15,2103.45,265625,1248.69,13988,2.11,15.92,2628.5,1.17,650.06,6.66,3.27,2.9,2.1
2018/7/16,48664,6113.3,2069.3,257215,1230.04,13433.9,2.04,15.52,2575.3,1.17,656.08,6.73,3.26,2.9,2.1
2018/7/23,49570,6232.85,2053.15,253400,1225.3,13564.8,2.13,15.44,2629.3,1.17,650.52,6.79,3.27,2.9,2.1
2018/7/30,49614,6187.7,2035.65,252945,1216.54,13578.25,2.09,15.39,2640.85,1.17,641.43,6.82,3.26,2.78,2.22
2018/8/6,49510,6149.8,2052.3,250650,1210.89,13773,2.04,15.35,2619.85,1.15,647.17,6.84,3.26,2.7,2.3
2018/8/13,48824,5936.35,2026.6,256630,1184.56,13231.3,2,14.76,2393.1,1.14,665.68,6.89,3.3,2.7,2.3
2018/8/20,48562,6003.6,2045.3,263475,1194.36,13385.7,2.05,14.69,2458.75,1.16,664.75,6.84,3.29,2.7,2.3
2018/8/27,48792,6070.35,2112.23,267005,1204.01,13275.7,2.11,14.65,2507.4,1.17,669.84,6.82,3.29,2.7,2.3
2018/9/3,48081,5882.85,2032.95,257600,1196.99,12412.4,1.99,14.21,2438.5,1.16,687.36,6.83,3.31,2.3,2.5
2018/9/10,48228,5936.15,2024.4,231420,1198.98,12422,1.99,14.14,2340,1.16,690.06,6.86,3.32,2.3,2.5
2018/9/17,49424,6107,2013.2,217055,1201.87,12519.6,2.01,14.23,2397.15,1.17,676.19,6.85,3.3,2.3,2.5
2018/9/24,50431,6277.05,2035.3,210095,1193.54,12687.4,2.07,14.36,2563.05,1.17,663.83,6.87,3.3,2.3,2.5
2018/10/1,50300,6252.85,2133.3,193130,1198.38,12489.1,2.12,14.59,2682.5,1.15,666.78,6.87,3.31,2.5,2.5
2018/10/8,50520,6260.55,2039.75,174005,1202.52,12642.6,2.01,14.42,2663.95,1.15,681.98,6.92,3.32,2.5,2.5
2018/10/15,50406,6221.35,2014.3,159805,1224.86,12427.8,1.93,14.61,2665.2,1.15,674.72,6.92,3.33,2.5,2.5
2018/10/22,50132,6217.05,1981.85,150695,1230.28,12157.1,1.84,14.64,2708.8,1.14,685.67,6.94,3.33,2.5,2.5
2018/10/29,49364,6142.6,1950.85,147410,1226.67,11676.1,1.77,14.52,2606.2,1.14,691.07,6.94,3.36,2.38,2.38
2018/11/5,49670,6162.5,1957.65,174185,1223.51,11645.1,1.66,14.45,2555.6,1.14,679.28,6.93,3.36,2.2,2.2
2018/11/12,49166,6138.25,1921.15,166440,1209.42,11256.4,1.58,14.15,2601.8,1.13,680.79,6.95,3.37,2.2,2.2
2018/11/19,49732,6258.05,1932.75,144605,1224.14,10967.55,1.49,14.38,2658,1.14,669.83,6.94,3.37,2.2,2.2
2018/11/26,49520,6218.95,1935.95,136940,1220.82,10869.9,1.43,14.21,2566.4,1.13,673.56,6.95,3.37,2.2,2.2
2018/12/3,49770,6185.7,1960.55,128395,1238.24,11008.4,1.45,14.48,2706.9,1.14,672.3,6.86,3.37,1.9,1.9
2018/12/10,49344,6138,1920.05,120980,1242.43,10788.6,1.44,14.61,2632,1.14,680.94,6.89,3.35,1.9,1.9
2018/12/17,48678,5995.15,1922.1,126020,1250.58,10831.2,1.36,14.63,2583.3,1.14,690.53,6.89,3.35,1.9,1.9
2018/12/24,48254,5957.7,1873.05,130025,1272.38,10719,1.3,15.01,2533.4,1.14,692.41,6.88,3.36,1.9,1.9
2018/12/31,47740,5852.6,1841.98,132265,1285.51,10786.35,1.33,15.56,2489.35,1.14,691.34,6.87,3.36,1.66,1.74
2019/1/7,47486,5911.05,1839.65,132935,1288.17,11197.1,1.39,15.61,2518.6,1.15,677.39,6.81,3.34,1.6,1.7
2019/1/14,47398,5937.1,1833.35,134480,1289.4,11579.2,1.41,15.51,2517.85,1.14,671.65,6.77,3.32,1.6,1.7
2019/1/21,47456,5941.3,1879.6,145515,1286.25,11698.9,1.4,15.38,2628.8,1.14,671.66,6.78,3.33,1.6,1.7
2019/1/28,47682,6079.75,1872.55,148465,1314.55,12218.8,1.37,15.89,2712.45,1.14,662.01,6.72,3.34,1.58,1.66
2019/2/4,47970,6209.35,1880.8,149570,1311.46,12885.85,1.44,15.76,2735.9,1.14,654.71,6.74,3.32,1.5,1.5
2019/2/11,48098,6136.25,1833.4,146680,1311.54,12299.65,1.48,15.64,2615.5,1.13,662.08,6.77,3.33,1.5,1.5
2019/2/18,49184,6403.7,1855.55,138120,1331.23,12694.4,1.59,15.89,2701.3,1.13,654.78,6.74,3.32,1.5,1.5
2019/2/25,49950,6534.5,1889.7,130205,1316.36,12965.6,1.62,15.65,2782.55,1.14,652.36,6.69,3.3,1.58,1.66
2019/3/4,50018,6478.5,1845.5,119140,1288.55,13282.1,1.78,15.1,2767.05,1.13,662.95,6.71,3.3,1.9,2.3
2019/3/11,49788,6459.25,1859.45,133815,1300.15,12916.75,1.84,15.32,2853.4,1.13,668.84,6.71,3.3,1.9,2.3
2019/3/18,50146,6431.6,1896.9,179215,1308.87,12981.8,1.91,15.4,2855.8,1.13,667.94,6.71,3.29,1.9,2.3
2019/3/25,49444,6371.4,1877.9,171055,1305.7,12906.8,1.92,15.26,2931.95,1.13,680.4,6.72,3.31,1.9,2.3
2019/4/1,49367,6436.9,1869.25,180105,1290.64,13078.2,1.94,15.11,2972.9,1.12,667.58,6.72,3.3,2,2.5
2019/4/8,49346,6447.7,1843.75,196040,1298.4,13036.95,2.03,15.1,2959.3,1.13,662.82,6.71,3.29,2,2.5
2019/4/15,49540,6471.47,1845.12,190208.33,1277.68,12734.13,2.05,14.98,2905.43,1.13,661.49,6.7,3.29,2,2.5
2019/4/22,49214,6402.53,1844.98,190746.67,1277.06,12328.17,2.12,14.95,2858.67,1.12,670.66,6.72,3.32,2,2.5
2019/4/29,48688,6289.95,1793.95,229510,1277.85,12155.2,2.06,14.8,2885.9,1.12,678.2,6.73,3.3,1.88,2.62
2019/5/6,48046,6138.9,1774.75,223180,1283.1,11916.75,1.98,14.81,2771,1.12,683.5,6.79,3.31,1.8,2.7
2019/5/13,47754,6028.25,1810.65,195785,1291.42,12005.4,2.01,14.65,2743.25,1.12,693.36,6.89,3.32,1.8,2.7
2019/5/20,47414,5939.15,1761.9,187595,1278.56,12052.1,1.97,14.48,2705.15,1.12,695.5,6.91,3.34,1.8,2.7
2019/5/27,46968,5871.9,1769.85,196145,1287.46,12089.6,1.91,14.48,2702.6,1.12,705.82,6.9,3.35,1.8,2.7
2019/6/3,46328,5804.25,1747.2,211770,1330.94,11673.2,1.72,14.85,2611,1.13,694.78,6.91,3.34,1.6,2.7
2019/6/10,46570,5833.8,1748.25,226890,1334.3,11736.4,1.73,14.79,2611.1,1.13,696.36,6.92,3.33,1.6,2.7
2019/6/17,46680,5911.6,1747.25,248715,1366.53,11965.65,1.76,15.15,2585.7,1.13,690.48,6.89,3.32,1.6,2.7
2019/6/24,47100,5979.25,1781.1,242375,1414.18,12400.5,1.92,15.32,2597.1,1.14,679.39,6.87,3.29,1.6,2.7
2019/7/1,46722,5902.85,1771.95,265565,1407.85,12246,1.91,15.2,2479.5,1.13,680.02,6.87,3.29,1.8,2.8
2019/7/8,46261,5897.4,1807.95,293210,1406.07,12946.6,1.96,15.13,2402.85,1.13,684.16,6.88,3.28,1.8,2.8
2019/7/15,47072,5993.95,1828.9,294565,1423.84,14303,1.87,15.89,2457.3,1.12,682.43,6.88,3.28,1.8,2.8
2019/7/22,47250,5969.85,1790.2,294740,1420.25,14198,1.86,16.43,2435.1,1.11,692.23,6.88,3.29,1.8,2.8
2019/7/29,46900,5881.2,1765.95,290070,1431.15,14424.6,1.84,16.36,2426.8,1.11,703.38,6.9,3.31,1.76,2.8
2019/8/5,46334,5700.85,1733.95,276690,1488.04,15196.2,1.67,16.77,2276.1,1.12,713.67,7.04,3.38,1.7,2.8
2019/8/12,46566,5741.2,1749.2,290285,1513.17,16017.8,1.68,17.12,2267.2,1.11,709.12,7.04,3.39,1.7,2.8
2019/8/19,46506,5681.7,1749.9,329895,1506.37,15822.3,1.67,17.12,2251.1,1.11,713.49,7.07,3.38,1.7,2.8
2019/8/26,46528,5662.75,1727.85,334970,1531.37,16447.4,1.65,18.16,2255.25,1.11,721.28,7.15,3.39,1.7,2.8
2019/9/2,46868,5707.3,1742.7,324280,1530.79,17882.8,1.53,18.81,2296.95,1.1,719.68,7.15,3.38,1.7,3
2019/9/9,47486,5815.55,1782.75,304480,1493.86,18028.8,1.57,17.91,2348.2,1.1,711.41,7.1,3.33,1.7,3
2019/9/16,47586,5788.95,1765.45,295000,1502,17394.8,1.69,17.86,2334.15,1.1,714.53,7.09,3.35,1.7,3
2019/9/23,47238,5738,1733.55,275725,1511.88,17408.7,1.65,18.09,2327.55,1.09,724.6,7.12,3.36,1.7,3
2019/9/30,47246.67,5645.95,1704.4,271190,1492.18,17594.8,1.57,17.37,2357.7,1.1,722.38,7.15,3.38,1.78,3.64
2019/10/7,47013.33,5698.07,1737.2,284000,1497.31,17704.5,1.6,17.58,2367.3,1.1,719.04,7.13,3.37,1.8,3.8
2019/10/14,46950,5744.6,1721.6,274280,1489.23,16599.2,1.62,17.5,2465.55,1.11,713.11,7.08,3.35,1.8,3.8
2019/10/21,47260,5844.95,1718.15,266870,1494.4,16551.7,1.64,17.69,2518.55,1.11,724.96,7.07,3.34,1.8,3.8
2019/10/28,47326,5854.45,1756.5,259030,1500.5,16766,1.66,17.94,2570.4,1.11,734.03,7.05,3.34,1.86,3.94
2019/11/4,47272,5906.7,1819.2,243180,1482.08,16273.9,1.65,17.41,2544.4,1.11,743.25,7,3.34,2.1,4.5
2019/11/11,47121,5825.35,1764.3,226895,1462.89,15347.4,1.62,16.89,2472.2,1.1,781.68,7.01,3.38,2.1,4.5
2019/11/18,47094,5834.95,1744.65,220575,1468.22,14561.5,1.65,17.07,2343.25,1.11,789.23,7.03,3.38,2.1,4.5
2019/11/25,47324,5879.4,1767.8,212465,1458.02,14179,1.66,16.96,2310.95,1.1,807.52,7.03,3.39,2.1,4.5
2019/12/2,47318,5866.05,1775.6,201990,1469.95,13342.8,1.6,16.87,2243,1.11,789.97,7.04,3.38,2.3,4.5
2019/12/9,48676,6096,1756.17,181070,1469.08,13711.2,1.65,16.79,2239.75,1.11,770.57,7.01,3.38,2.3,4.5
2019/12/16,48988,6169.65,1764.3,160495,1476.77,14088.8,1.69,17.04,2312.2,1.11,755.5,7,3.33,2.3,4.5
2019/12/23,49188,6182.82,1790.32,148962.5,1502.45,14232.15,1.73,17.73,2291.7,1.11,748.93,7,3.31,2.3,4.5
2019/12/30,49133,6153.75,1785.27,145442.5,1527.23,14027.05,1.72,17.92,2307.27,1.12,752.65,6.97,3.31,2.42,5.04
2020/1/6,48790,6143.82,1781.85,138150,1561.98,13939.6,1.69,18.11,2377.9,1.11,767.99,6.94,3.32,2.5,5.4
2020/1/13,48966,6257.2,1784.25,127635,1551.88,13912.9,1.65,17.93,2412.5,1.11,773.4,6.88,3.32,2.5,5.4
2020/1/20,48292,6058.25,1794.15,172590,1562.3,13446.1,1.59,17.9,2412.5,1.11,773.16,6.91,3.32,2.5,5.4
2020/1/27,46380,5624.55,1727.85,183930,1577.86,12556.3,1.5,17.77,2227.35,1.1,792.05,6.94,3.35,2.5,5.4
2020/2/3,45406,5634.4,1683.8,176205,1564.43,12826.4,1.48,17.67,2178.35,1.1,783.57,6.99,3.37,2.3,5.2
2020/2/10,45492,5730.2,1702.73,167150,1573,12995.2,1.55,17.65,2141,1.09,792,6.98,3.38,2.3,5.2
2020/2/17,46002,5748.55,1689.7,164515,1611.2,12703.5,1.64,18.23,2119.1,1.08,799,7,3.38,2.3,5.2
2020/2/24,45338,5637.85,1676.15,196260,1633,12337.1,1.48,17.78,2014.5,1.09,812.22,7.01,3.42,2.3,5.2
2020/3/2,45010,5647.05,1698.5,207660,1642.24,12675,1.51,17.18,1976.7,1.12,818.83,6.94,3.44,1.5,4.3
2020/3/9,43594,5494.1,1661.82,189085,1613.71,12331.4,1.04,16.23,1959.45,1.13,840.59,6.98,3.51,1.5,4.3
2020/3/16,40596,4951.9,1616.55,218420,1499.53,11446.2,0.66,12.43,1858.2,1.09,859.12,7.05,3.55,1.5,4.3
2020/3/23,38284,4770.68,1513,225360,1609.12,11131.1,0.5,14.16,1830,1.09,842.46,7.08,3.47,1.5,4.3
2020/3/30,39392,4837.6,1472.16,220955,1603.03,11270.64,0.61,14.16,1873.25,1.09,858.23,7.09,3.44,0.78,3.7
2020/4/6,40843,4970.59,1436.66,242875,1665.5,11453.45,0.68,15.14,1898.72,1.09,846.13,7.05,3.37,0.3,3.3
2020/4/13,42036,5118.35,1466.53,261455,1711.18,11777.17,0.71,15.46,1920.13,1.09,850.95,7.06,3.4,0.3,3.3
2020/4/20,42024,5104.65,1469.03,263805,1710.27,12160.75,0.62,15.12,1891.3,1.08,857.29,7.08,3.38,0.3,3.3
2020/4/27,42911,5168.05,1461.97,255180,1702.33,12132.05,0.7,15.09,1919.4,1.09,840.09,7.07,3.38,0.26,3.12
2020/5/4,43131,5175.15,1445.55,245855,1701.51,12085.5,0.9,15.06,1955.95,1.08,834.93,7.07,3.4,0.1,2.4
2020/5/11,43538,5194.38,1443.02,251975,1716.93,12106.45,0.92,15.78,1985.35,1.08,822.33,7.09,3.43,0.1,2.4
2020/5/18,43992,5331.91,1476.48,273575,1736.53,12413.35,1.04,17.23,2017.35,1.09,810.19,7.11,3.42,0.1,2.4
2020/5/25,44104,5301.05,1504.72,266775,1718.56,12198.15,1.02,17.33,1964.45,1.1,807.37,7.14,3.44,0.1,2.4
2020/6/1,44970,5529.17,1536.29,250880,1711.58,12753.55,1.13,17.82,2027.41,1.12,778.1,7.11,3.42,0.6,2.5
2020/6/8,46702,5764.4,1580.46,234820,1721.01,12788.07,1.17,17.68,2003.15,1.13,778.59,7.07,3.44,0.6,2.5
2020/6/15,47128,5746.2,1575.21,244005,1727.88,12799,1.23,17.44,2019.63,1.12,800.05,7.09,3.49,0.6,2.5
2020/6/22,48230,5889.25,1571.5,227045,1763.09,12555.75,1.23,17.72,2044.75,1.13,817.26,7.07,3.51,0.6,2.5
2020/6/29,49264,6020.25,1595.45,213320,1774.48,12828.3,1.22,17.99,2033.3,1.12,810.04,7.07,3.53,0.84,2.62
2020/7/6,50040,6253.83,1626.27,190395,1797.63,13363.85,1.27,18.51,2110.52,1.13,790.74,7,3.52,1,2.7
2020/7/13,52242,6469.6,1644.12,165635,1804.89,13461.55,1.25,19.23,2203.85,1.14,786.44,7,3.5,1,2.7
2020/7/20,51994,6508.39,1654.05,145590,1863.51,13395.1,1.27,21.91,2216.3,1.16,773.4,7,3.51,1,2.7
2020/7/27,51914,6452.45,1679.9,132260,1960.1,13704.9,1.24,24.21,2269.33,1.18,761.5,7,3.51,1,2.7
2020/8/3,51450,6446.9,1730.37,122635,2026.22,14224.55,1.22,26.9,2366,1.18,773,6.96,3.54,1.3,2.4
2020/8/10,50384,6370.25,1737.67,114130,1950.12,14220.34,1.23,26.59,2372.24,1.18,796.13,6.95,3.57,1.3,2.4
2020/8/17,51682,6571.95,1738.37,106330,1960.46,14617.7,1.28,27.13,2448.6,1.19,792.67,6.92,3.57,1.3,2.4
2020/8/24,51526,6606.65,1742.62,93115,1941.47,15088.4,1.35,26.89,2453.35,1.18,784.07,6.89,3.57,1.3,2.4
2020/8/31,52026,6685.12,1759.65,85825,1948.38,15383.35,1.22,27.43,2497.05,1.19,773.06,6.84,3.54,1.38,1.84
2020/9/7,52258,6744.95,1749.54,75810,1938.32,14940.8,1.11,26.81,2420.49,1.18,771.11,6.83,3.55,1.4,1.7
2020/9/14,52140,6808.93,1752.5,77995,1952.59,15086.35,1.18,27.04,2491.05,1.18,762.49,6.77,3.54,1.4,1.7
2020/9/21,51604,6642.88,1721.1,76790,1880.43,14375.3,1.19,23.57,2396.25,1.17,780.94,6.81,3.57,1.4,1.7
2020/9/28,51247.71,6544.6,1731.6,146030,1893.65,14408.4,1.18,23.71,2367.43,1.17,785.79,6.79,3.6,1.32,1.22
2020/10/5,51504.29,6628,1772.93,156215,1900.25,14689.85,1.21,24.03,2350.72,1.18,796.82,6.72,3.58,1.2,0.5
2020/10/12,51626,6714.84,1843.83,165965,1904.51,15277.3,1.18,24.38,2410.6,1.17,798.29,6.73,3.58,1.2,0.5
2020/10/19,52038,6874.95,1830.73,181930,1908.08,15783.85,1.16,24.67,2530.5,1.18,782.2,6.68,3.6,1.2,0.5
2020/10/26,51726,6738.68,1812.16,174925,1886.24,15565.35,1.09,23.77,2523.65,1.17,773.16,6.71,3.61,1.2,0.5
2020/11/2,51620,6833.25,1889.57,172430,1921.1,15319.98,1.09,24.62,2578.93,1.17,757.4,6.65,3.6,1.2,-0.5
2020/11/9,52124,6912.91,1897.95,167985,1873.97,15847.5,1.16,24.29,2618.12,1.18,758.63,6.62,3.62,1.2,-0.5
2020/11/16,53216,7109.6,1968.47,160350,1875.05,15880.85,1.16,24.35,2725.3,1.19,762,6.57,3.6,1.2,-0.5
2020/11/23,54890,7319.8,1965,153375,1809.29,16147.15,1.26,23.23,2743.15,1.19,769.04,6.58,3.6,1.2,-0.5
2020/11/30,57448,7668.6,2036.04,149680,1819.98,16083.1,1.25,23.79,2743.84,1.21,753.99,6.56,3.6,1.36,0.06
2020/12/7,57418,7739.04,2008.47,148695,1849.2,16782.61,1.29,24.17,2791.67,1.21,740.08,6.54,3.59,1.4,0.2
2020/12/14,58154,7834.45,2030.13,134185,1861.74,17461.05,1.36,25.07,2816.59,1.22,730.26,6.54,3.59,1.4,0.2
2020/12/21,58320,7783.67,2007.68,118830,1873.45,16885.7,1.37,25.68,2795.85,1.22,717.99,6.54,3.61,1.4,0.2
2020/12/28,58336,7797.04,1987.59,111157.5,1888.24,16717.95,1.4,26.33,2747.66,1.22,711.24,6.52,3.62,1.4,0.2

print("initial_n:",initial_n, "number_data:", number_data)

x_train_scaled = x_train_scaled[:100]
y_train_scaled = y_train_scaled[:100]


initial_x = x_train_scaled[:x_train_scaled.shape[1]+1]
initial_y = y_train_scaled[:x_train_scaled.shape[1]+1]

x_train_scaled = x_train_scaled[x_train_scaled.shape[1]+1:]
y_train_scaled = y_train_scaled[x_train_scaled.shape[1]+1:]

network = Network(4, initial_x, initial_y)
initializing(network, initial_x, initial_y)

# network.setData(x_train_scaled[:18], y_train_scaled[:18])
# print(network.forward()[1])

#for i in range(initial_n, number_data): 

for i in range(0, x_train_scaled.shape[0]):
    print(x_train_scaled.shape[0])
    print("The data number: %d"%i)
    
    sorted_index = selecting(network, x_train_scaled, y_train_scaled)
#     print("Selecting module finish!")
    
    network.addData(x_train_scaled[sorted_index[0]], y_train_scaled[sorted_index[0]])
    x_train_scaled = np.delete(x_train_scaled, sorted_index[0], 0)
    y_train_scaled = np.delete(y_train_scaled, sorted_index[0], 0)
    
    yo, loss, tape = network.forward()
    
    print(tf.math.abs(yo-network.y))
    
    if tf.reduce_all(tf.math.abs(yo-network.y) <= network.threshold_for_error):
        network.acceptable = True
        network = reorganizing(network)
 
    else:
        
        network.acceptable = False
        network_pre = copy.deepcopy(network)
        network = matching(network)
        
        if network.acceptable:
            network = reorganizing(network)
 
        else:
            network = network_pre
            cramming(network)
            network = reorganizing(network)

#     print(network.b1)
    print(network.acceptable)

initial_n: 20 number_data: 377
81
The data number: 0
First: (0.04087187, 0)
Selecting module finish!
tf.Tensor(
[[0.00000376 0.00000101 0.00000083 0.        ]
 [0.00000203 0.00000203 0.00000036 0.00000018]
 [0.00000662 0.00000179 0.00000006 0.00000006]
 [0.00000101 0.00000089 0.00000006 0.00000024]
 [0.00000167 0.00000328 0.00000036 0.00000018]
 [0.00000322 0.00000274 0.00000089 0.00000077]
 [0.00000542 0.00000399 0.0000003  0.00000012]
 [0.00000131 0.0000034  0.         0.00000054]
 [0.00000119 0.00000644 0.0000003  0.00000042]
 [0.00000149 0.00000054 0.0000003  0.00000066]
 [0.00000703 0.00000638 0.0000003  0.00000048]
 [0.00000715 0.00000376 0.00000012 0.00000006]
 [0.00000644 0.00000536 0.00000006 0.00000024]
 [0.00000423 0.0000059  0.00000012 0.        ]
 [0.00000286 0.00000012 0.00000012 0.        ]
 [0.00000066 0.00000417 0.00000012 0.        ]
 [0.00000149 0.00000942 0.00000036 0.        ]
 [0.00000471 0.00000727 0.00000012 0.00000024]
 [0.00000232 0.00000727 0.00000012 0.00000

In [ ]:
yo, loss, tape = network.forward()
plt.plot(sc.inverse_transform(yo)[:,0], label="LLAAT")
plt.plot(sc.inverse_transform(network.y)[:,0], label="Actual")
plt.legend()

In [ ]:
#print(x_test_scaled.shape)

#x_test_scaled = tf.cast(x_test_scaled, tf.float32)
#activation_value = tf.nn.relu((tf.matmul(x_test_scaled, network.W1)+network.b1))
#forecast_value = tf.matmul(activation_value,network.Wo)+network.bo

forecast_value = network.forecast(x_test_scaled)

#
plt.plot(sc.inverse_transform(forecast_value)[:,0])
plt.plot(y_test_data[:,0])
#network.forecast(x_test_scaled)

In [ ]:
# def forecast(self, x_test_scaled):
    
#     x_test_scaled = tf.cast(x_test_scaled, tf.float32)
#     forecast_value = tf.nn.relu((tf.matmul(x_test_scaled, self.W1)+self.b1))
#     return forecast_value